In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from pathlib import Path


# Build robust path to data folder (notebooks and data are siblings)
DATA_DIR = Path.cwd().parent / 'data'
SUB_DIR = Path.cwd().parent / 'submissions'
train_path = DATA_DIR / 'train.csv'
test_path = DATA_DIR / 'test.csv'


# Load the datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)  # This is for final predictions (no 'W' column)

# Display basic information about the datasets
print(f"Training set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")
print(f"'W' column in train dataset: {'W' in train_df.columns}")
print(f"'W' column in test dataset: {'W' in test_df.columns}")

Training set shape: (1812, 51)
Test set shape: (453, 45)
'W' column in train dataset: True
'W' column in test dataset: False


In [2]:
# # Perform analysis for outlier detection and perform outlier handling

# import seaborn as sns
# from scipy import stats

# print("COMPREHENSIVE OUTLIER ANALYSIS AND HANDLING")
# print("=" * 50)

# # Get numerical columns but exclude categorical/engineered features that shouldn't be treated as numerical
# numerical_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()

# # Exclude target, ID, and categorical/engineered columns
# exclude_cols = ['W', 'ID', 'yearID', 'year_label', 'decade_label', 'win_bins']
# numerical_cols = [col for col in numerical_cols if col not in exclude_cols]

# print(f"Excluding from outlier analysis: {exclude_cols}")
# print(f"Remaining numerical features for outlier analysis: {len(numerical_cols)}")

# # 1. Identify outliers using multiple methods
# print(f"\n1. OUTLIER DETECTION ON {len(numerical_cols)} FEATURES")
# print("-" * 50)

# outlier_summary = {}
# for col in numerical_cols:
#     if col in train_df.columns and train_df[col].std() > 0:
#         # IQR method
#         Q1 = train_df[col].quantile(0.25)
#         Q3 = train_df[col].quantile(0.75)
#         IQR = Q3 - Q1
#         lower_bound = Q1 - 1.5 * IQR
#         upper_bound = Q3 + 1.5 * IQR
#         iqr_outliers = ((train_df[col] < lower_bound) | (train_df[col] > upper_bound)).sum()
        
#         # Z-score method  
#         z_scores = np.abs(stats.zscore(train_df[col].dropna()))
#         z_outliers = (z_scores > 3).sum()
        
#         outlier_summary[col] = {
#             'iqr_outliers': iqr_outliers,
#             'z_outliers': z_outliers,
#             'outlier_rate': max(iqr_outliers, z_outliers) / len(train_df)
#         }

# # Sort by outlier rate
# sorted_outliers = sorted(outlier_summary.items(), 
#                         key=lambda x: x[1]['outlier_rate'], reverse=True)

# print("Top 10 features with highest outlier rates:")
# for col, stats_dict in sorted_outliers[:10]:
#     rate = stats_dict['outlier_rate']
#     iqr_count = stats_dict['iqr_outliers'] 
#     z_count = stats_dict['z_outliers']
#     print(f"  {col:>15}: {rate*100:.1f}% (IQR: {iqr_count}, Z-score: {z_count})")

# # 2. Select features for outlier handling (>5% outlier rate)
# features_to_handle = [col for col, stats_dict in sorted_outliers 
#                      if stats_dict['outlier_rate'] > 0.05]

# print(f"\n2. FEATURES SELECTED FOR OUTLIER HANDLING")
# print("-" * 50)
# print(f"Features with >5% outlier rate: {len(features_to_handle)}")
# for feature in features_to_handle:
#     rate = outlier_summary[feature]['outlier_rate']
#     print(f"  {feature}: {rate*100:.1f}%")

# if features_to_handle:
#     print(f"\n3. APPLYING OUTLIER HANDLING")
#     print("-" * 50)
    
#     # Store original for comparison
#     train_df_original = train_df.copy()
    
#     # Handle each problematic feature
#     bounds_applied = {}
    
#     # Filter to only handle columns that exist in both train AND test datasets
#     valid_features_to_handle = [col for col in features_to_handle 
#                                if col in train_df.columns and col in test_df.columns]
    
#     print(f"Processing {len(valid_features_to_handle)} features that exist in both train and test sets")
    
#     for col in valid_features_to_handle:
#         # Calculate bounds on TRAINING data only
#         lower_bound = train_df[col].quantile(0.01)
#         upper_bound = train_df[col].quantile(0.99)
        
#         # Store bounds for reference
#         bounds_applied[col] = {
#             'lower': lower_bound,
#             'upper': upper_bound,
#             'original_range': f"{train_df[col].min():.2f} to {train_df[col].max():.2f}"
#         }
        
#         # Apply SAME bounds to both datasets
#         train_df[col] = train_df[col].clip(lower_bound, upper_bound)
#         test_df[col] = test_df[col].clip(lower_bound, upper_bound)  # Same bounds!
        
#         bounds_applied[col]['new_range'] = f"{train_df[col].min():.2f} to {train_df[col].max():.2f}"
    
#     # Report on any features that were excluded from processing
#     excluded_features = [col for col in features_to_handle if col not in valid_features_to_handle]
#     if excluded_features:
#         print(f"\n⚠️  Skipped {len(excluded_features)} features not present in test set:")
#         for col in excluded_features:
#             print(f"    - {col} (train-only feature)")
    
#     print(f"\n✅ Outlier handling complete!")
#     print(f"Features processed: {len(bounds_applied)}")
#     print(f"All rows preserved: Train {train_df.shape[0]}, Test {test_df.shape[0]}")
    
#     print(f"\n4. IMPACT SUMMARY")
#     print("-" * 50)
#     for col, bounds in bounds_applied.items():
#         print(f"{col}:")
#         print(f"  Bounds applied: {bounds['lower']:.2f} to {bounds['upper']:.2f}")
#         print(f"  Before: {bounds['original_range']}")
#         print(f"  After:  {bounds['new_range']}")
#         print()

# else:
#     print(f"\n✅ No features require outlier handling (all <5% outlier rate)")



In [3]:
# Create derived features for both train and test sets

# R_per_game: Runs per game
# RA_per_game: Runs allowed per game
train_df['R_per_game'] = train_df['R'] / train_df['G']
train_df['RA_per_game'] = train_df['RA'] / train_df['G']
test_df['R_per_game'] = test_df['R'] / test_df['G']
test_df['RA_per_game'] = test_df['RA'] / test_df['G']

print(f"\nCreated derived features: R_per_game, RA_per_game")
print(f"Train - R_per_game range: {train_df['R_per_game'].min():.3f} to {train_df['R_per_game'].max():.3f}")
print(f"Train - RA_per_game range: {train_df['RA_per_game'].min():.3f} to {train_df['RA_per_game'].max():.3f}")
print(f"Test - R_per_game range: {test_df['R_per_game'].min():.3f} to {test_df['R_per_game'].max():.3f}")
print(f"Test - RA_per_game range: {test_df['RA_per_game'].min():.3f} to {test_df['RA_per_game'].max():.3f}")

# Expected Wins of Season = G × (R²) / (R² + RA²)
train_df['Expected_Wins'] = train_df['G'] * (train_df['R_per_game'] ** 2) / ((train_df['R_per_game'] ** 2) + (train_df['RA_per_game'] ** 2))
test_df['Expected_Wins'] = test_df['G'] * (test_df['R_per_game'] ** 2) / ((test_df['R_per_game'] ** 2) + (test_df['RA_per_game'] ** 2))
# train_df['Expected_Wins'] = train_df['G'] * (train_df['R'] ** 2) / ((train_df['R'] ** 2) + (train_df['RA'] ** 2))
# test_df['Expected_Wins'] = test_df['G'] * (test_df['R'] ** 2) / ((test_df['R'] ** 2) + (test_df['RA'] ** 2))
print(f"\nCreated derived feature: Expected_Wins")   
print(f"Train - Expected_Wins range: {train_df['Expected_Wins'].min():.3f} to {train_df['Expected_Wins'].max():.3f}")
print(f"Test - Expected_Wins range: {test_df['Expected_Wins'].min():.3f} to {test_df['Expected_Wins'].max():.3f}")

# Times getting on base
train_df['Times_On_Base'] = train_df['H'] + train_df['BB']
test_df['Times_On_Base'] = test_df['H'] + test_df['BB']

print(f"\nCreated derived feature: Times_On_Base")
print(f"Train - Times_On_Base range: {train_df['Times_On_Base'].min():.3f} to {train_df['Times_On_Base'].max():.3f}")
print(f"Test - Times_On_Base range: {test_df['Times_On_Base'].min():.3f} to {test_df['Times_On_Base'].max():.3f}")

# BB Rate (Walk Percentage) - BB / AB + BB
train_df['BB_Rate'] = train_df['BB'] / (train_df['AB'] + train_df['BB'])
test_df['BB_Rate'] = test_df['BB'] / (test_df['AB'] + test_df['BB'])

print(f"\nCreated derived feature: BB_Rate")
print(f"Train - BB_Rate range: {train_df['BB_Rate'].min():.3f} to {train_df['BB_Rate'].max():.3f}") 
print(f"Test - BB_Rate range: {test_df['BB_Rate'].min():.3f} to {test_df['BB_Rate'].max():.3f}")

# Home Run Rate - HR / AB
train_df['HR_Rate'] = train_df['HR'] / train_df['AB']
test_df['HR_Rate'] = test_df['HR'] / test_df['AB']

print(f"\nCreated derived feature: HR_Rate")
print(f"Train - HR_Rate range: {train_df['HR_Rate'].min():.3f} to {train_df['HR_Rate'].max():.3f}")
print(f"Test - HR_Rate range: {test_df['HR_Rate'].min():.3f} to {test_df['HR_Rate'].max():.3f}")

# On-Base Percentage (OBP) - (H + BB) / (AB + BB)
train_df['OBP'] = (train_df['H'] + train_df['BB']) / (train_df['AB'] + train_df['BB'])
test_df['OBP'] = (test_df['H'] + test_df['BB']) / (test_df['AB'] + test_df['BB'])

print(f"\nCreated derived feature: OBP")
print(f"Train - OBP range: {train_df['OBP'].min():.3f} to {train_df['OBP'].max():.3f}") 
print(f"Test - OBP range: {test_df['OBP'].min():.3f} to {test_df['OBP'].max():.3f}")

# Slugging Percentage (SLG)
# Singles = H - (2B + 3B + HR)
# Total Bases = Singles + (2 * 2B) + (3 * 3B) + (4 * HR)
# SLG = Total Bases / AB
Singles_train = train_df['H'] - (train_df['2B'] + train_df['3B'] + train_df['HR'])
Total_Bases_train = Singles_train + (2 * train_df['2B']) + (3 * train_df['3B']) + (4 * train_df['HR'])
train_df['SLG'] = Total_Bases_train / train_df['AB']  

Singles_test = test_df['H'] - (test_df['2B'] + test_df['3B'] + test_df['HR'])
Total_Bases_test = Singles_test + (2 * test_df['2B']) + (3 * test_df['3B']) + (4 * test_df['HR'])
test_df['SLG'] = Total_Bases_test / test_df['AB']

print(f"\nCreated derived feature: SLG")
print(f"Train - SLG range: {train_df['SLG'].min():.3f} to {train_df['SLG'].max():.3f}") 
print(f"Test - SLG range: {test_df['SLG'].min():.3f} to {test_df['SLG'].max():.3f}")    

# Combined On-Base Plus Slugging (OPS) - OBP + SLG
train_df['OPS'] = train_df['OBP'] + train_df['SLG']
test_df['OPS'] = test_df['OBP'] + test_df['SLG']

print(f"\nCreated derived feature: OPS")
print(f"Train - OPS range: {train_df['OPS'].min():.3f} to {train_df['OPS'].max():.3f}") 
print(f"Test - OPS range: {test_df['OPS'].min():.3f} to {test_df['OPS'].max():.3f}")

# Time on Base Allowed - HA + BBA
train_df['Times_On_Base_Allowed'] = train_df['HA'] + train_df['BBA']
test_df['Times_On_Base_Allowed'] = test_df['HA'] + test_df['BBA']

print(f"\nCreated derived feature: Times_On_Base_Allowed")
print(f"Train - Times_On_Base_Allowed range: {train_df['Times_On_Base_Allowed'].min():.3f} to {train_df['Times_On_Base_Allowed'].max():.3f}")
print(f"Test - Times_On_Base_Allowed range: {test_df['Times_On_Base_Allowed'].min():.3f} to {test_df['Times_On_Base_Allowed'].max():.3f}")

# WHIP (Walks plus Hits per Inning Pitched)
# Inings Pitched = IPouts / 3
# Times_On_Base_Per_Inning = Times_On_Base_Allowed / Inings_Pitched
train_df['Innings_Pitched'] = train_df['IPouts'] / 3
train_df['WHIP'] = train_df['Times_On_Base_Allowed'] / train_df['Innings_Pitched']
test_df['Innings_Pitched'] = test_df['IPouts'] / 3
test_df['WHIP'] = test_df['Times_On_Base_Allowed'] / test_df['Innings_Pitched']

print(f"\nCreated derived feature: WHIP")
print(f"Train - WHIP range: {train_df['WHIP'].min():.3f} to {train_df['WHIP'].max():.3f}")
print(f"Test - WHIP range: {test_df['WHIP'].min():.3f} to {test_df['WHIP'].max():.3f}")

# K/9 (Strikeouts per 9 Innings) - SOA / Innings_Pitched * 9
train_df['K_per_9'] = (train_df['SOA'] / train_df['Innings_Pitched']) * 9
test_df['K_per_9'] = (test_df['SOA'] / test_df['Innings_Pitched']) * 9  

print(f"\nCreated derived feature: K_per_9")
print(f"Train - K_per_9 range: {train_df['K_per_9'].min():.3f} to {train_df['K_per_9'].max():.3f}")
print(f"Test - K_per_9 range: {test_df['K_per_9'].min():.3f} to {test_df['K_per_9'].max():.3f}")

# HR/9 (Home Runs Allowed per 9 Innings) - HRA / Innings_Pitched * 9
train_df['HR_per_9'] = (train_df['HRA'] / train_df['Innings_Pitched']) * 9
test_df['HR_per_9'] = (test_df['HRA'] / test_df['Innings_Pitched']) * 9

print(f"\nCreated derived feature: HR_per_9")
print(f"Train - HR_per_9 range: {train_df['HR_per_9'].min():.3f} to {train_df['HR_per_9'].max():.3f}")
print(f"Test - HR_per_9 range: {test_df['HR_per_9'].min():.3f} to {test_df['HR_per_9'].max():.3f}")

# Run Environment Idex (REI) - (R + RA) / G / mlb_rpg
train_df['REI'] = (train_df['R'] + train_df['RA']) / train_df['G'] / train_df['mlb_rpg']
test_df['REI'] = (test_df['R'] + test_df['RA']) / test_df['G'] / test_df['mlb_rpg']
print(f"\nCreated derived feature: REI")
print(f"Train - REI range: {train_df['REI'].min():.3f} to {train_df['REI'].max():.3f}")
print(f"Test - REI range: {test_df['REI'].min():.3f} to {test_df['REI'].max():.3f}")    

# Power Environement Index (PEI) -  (HR + HRA) / G / (mlb_rpg * avg_hr_rate)
avg_hr_rate = train_df['HR_Rate'].mean()
train_df['PEI'] = (train_df['HR'] + train_df['HRA']) / train_df['G'] / (train_df['mlb_rpg'] * avg_hr_rate)
test_df['PEI'] = (test_df['HR'] + test_df['HRA']) / test_df['G'] / (test_df['mlb_rpg'] * avg_hr_rate)
print(f"\nCreated derived feature: PEI")
print(f"Train - PEI range: {train_df['PEI'].min():.3f} to {train_df['PEI'].max():.3f}")
print(f"Test - PEI range: {test_df['PEI'].min():.3f} to {test_df['PEI'].max():.3f}") 

# Era adjusted OBP, SLG, OPS, WHIP, K_per_9, HR_per_9, BB_Rate, HR_Rate
# Historical average runs per game (RPG) for MLB
historical_avg_rpg_train = train_df['mlb_rpg'].mean()
historical_avg_rpg_test = test_df['mlb_rpg'].mean()
# historical_avg_rpg_train = 4.4
# historical_avg_rpg_test = 4.4

# Era adjusted OBP
train_df['Era_Adjusted_OBP'] = train_df['OBP'] * (historical_avg_rpg_train / train_df['mlb_rpg'])
test_df['Era_Adjusted_OBP'] = test_df['OBP'] * (historical_avg_rpg_test / test_df['mlb_rpg'])
print(f"\nCreated derived feature: Era_Adjusted_OBP")
print(f"Train - Era_Adjusted_OBP range: {train_df['Era_Adjusted_OBP'].min():.3f} to {train_df['Era_Adjusted_OBP'].max():.3f}") 
print(f"Test - Era_Adjusted_OBP range: {test_df['Era_Adjusted_OBP'].min():.3f} to {test_df['Era_Adjusted_OBP'].max():.3f}") 

# Era adjusted SLG
train_df['Era_Adjusted_SLG'] = train_df['SLG'] * (historical_avg_rpg_train / train_df['mlb_rpg'])
test_df['Era_Adjusted_SLG'] = test_df['SLG'] * (historical_avg_rpg_test / test_df['mlb_rpg'])
print(f"\nCreated derived feature: Era_Adjusted_SLG")
print(f"Train - Era_Adjusted_SLG range: {train_df['Era_Adjusted_SLG'].min():.3f} to {train_df['Era_Adjusted_SLG'].max():.3f}") 
print(f"Test - Era_Adjusted_SLG range: {test_df['Era_Adjusted_SLG'].min():.3f} to {test_df['Era_Adjusted_SLG'].max():.3f}") 

# Era adjusted OPS
train_df['Era_Adjusted_OPS'] = train_df['OPS'] * (historical_avg_rpg_train / train_df['mlb_rpg'])
test_df['Era_Adjusted_OPS'] = test_df['OPS'] * (historical_avg_rpg_test / test_df['mlb_rpg'])
print(f"\nCreated derived feature: Era_Adjusted_OPS")
print(f"Train - Era_Adjusted_OPS range: {train_df['Era_Adjusted_OPS'].min():.3f} to {train_df['Era_Adjusted_OPS'].max():.3f}") 
print(f"Test - Era_Adjusted_OPS range: {test_df['Era_Adjusted_OPS'].min():.3f} to {test_df['Era_Adjusted_OPS'].max():.3f}")

# Era adjusted WHIP
train_df['Era_Adjusted_WHIP'] = train_df['WHIP'] * (historical_avg_rpg_train / train_df['mlb_rpg'])
test_df['Era_Adjusted_WHIP'] = test_df['WHIP'] * (historical_avg_rpg_test / test_df['mlb_rpg'])
print(f"\nCreated derived feature: Era_Adjusted_WHIP")
print(f"Train - Era_Adjusted_WHIP range: {train_df['Era_Adjusted_WHIP'].min():.3f} to {train_df['Era_Adjusted_WHIP'].max():.3f}")
print(f"Test - Era_Adjusted_WHIP range: {test_df['Era_Adjusted_WHIP'].min():.3f} to {test_df['Era_Adjusted_WHIP'].max():.3f}")

# Era adjusted K_per_9
train_df['Era_Adjusted_K_per_9'] = train_df['K_per_9'] * (historical_avg_rpg_train / train_df['mlb_rpg'])
test_df['Era_Adjusted_K_per_9'] = test_df['K_per_9'] * (historical_avg_rpg_test / test_df['mlb_rpg'])
print(f"\nCreated derived feature: Era_Adjusted_K_per_9")
print(f"Train - Era_Adjusted_K_per_9 range: {train_df['Era_Adjusted_K_per_9'].min():.3f} to {train_df['Era_Adjusted_K_per_9'].max():.3f}")
print(f"Test - Era_Adjusted_K_per_9 range: {test_df['Era_Adjusted_K_per_9'].min():.3f} to {test_df['Era_Adjusted_K_per_9'].max():.3f}") 

# Era adjusted HR_per_9
train_df['Era_Adjusted_HR_per_9'] = train_df['HR_per_9'] * (historical_avg_rpg_train / train_df['mlb_rpg'])
test_df['Era_Adjusted_HR_per_9'] = test_df['HR_per_9'] * (historical_avg_rpg_test / test_df['mlb_rpg'])
print(f"\nCreated derived feature: Era_Adjusted_HR_per_9")
print(f"Train - Era_Adjusted_HR_per_9 range: {train_df['Era_Adjusted_HR_per_9'].min():.3f} to {train_df['Era_Adjusted_HR_per_9'].max():.3f}")
print(f"Test - Era_Adjusted_HR_per_9 range: {test_df['Era_Adjusted_HR_per_9'].min():.3f} to {test_df['Era_Adjusted_HR_per_9'].max():.3f}")

# Era adjusted BB_Rate
train_df['Era_Adjusted_BB_Rate'] = train_df['BB_Rate'] * (historical_avg_rpg_train / train_df['mlb_rpg'])
test_df['Era_Adjusted_BB_Rate'] = test_df['BB_Rate'] * (historical_avg_rpg_test / test_df['mlb_rpg'])
print(f"\nCreated derived feature: Era_Adjusted_BB_Rate")
print(f"Train - Era_Adjusted_BB_Rate range: {train_df['Era_Adjusted_BB_Rate'].min():.3f} to {train_df['Era_Adjusted_BB_Rate'].max():.3f}")
print(f"Test - Era_Adjusted_BB_Rate range: {test_df['Era_Adjusted_BB_Rate'].min():.3f} to {test_df['Era_Adjusted_BB_Rate'].max():.3f}") 

# Era adjusted HR_Rate
train_df['Era_Adjusted_HR_Rate'] = train_df['HR_Rate'] * (historical_avg_rpg_train / train_df['mlb_rpg'])
test_df['Era_Adjusted_HR_Rate'] = test_df['HR_Rate'] * (historical_avg_rpg_test / test_df['mlb_rpg'])
print(f"\nCreated derived feature: Era_Adjusted_HR_Rate")
print(f"Train - Era_Adjusted_HR_Rate range: {train_df['Era_Adjusted_HR_Rate'].min():.3f} to {train_df['Era_Adjusted_HR_Rate'].max():.3f}")
print(f"Test - Era_Adjusted_HR_Rate range: {test_df['Era_Adjusted_HR_Rate'].min():.3f} to {test_df['Era_Adjusted_HR_Rate'].max():.3f}")



Created derived features: R_per_game, RA_per_game
Train - R_per_game range: 2.409 to 6.884
Train - RA_per_game range: 2.458 to 7.686
Test - R_per_game range: 2.783 to 6.896
Test - RA_per_game range: 2.867 to 6.865

Created derived feature: Expected_Wins
Train - Expected_Wins range: 35.860 to 119.963
Test - Expected_Wins range: 40.352 to 107.111

Created derived feature: Times_On_Base
Train - Times_On_Base range: 1367.000 to 2415.000
Test - Times_On_Base range: 1453.000 to 2327.000

Created derived feature: BB_Rate
Train - BB_Rate range: 0.051 to 0.136
Test - BB_Rate range: 0.052 to 0.123

Created derived feature: HR_Rate
Train - HR_Rate range: 0.001 to 0.047
Test - HR_Rate range: 0.001 to 0.045

Created derived feature: OBP
Train - OBP range: 0.262 to 0.382
Test - OBP range: 0.267 to 0.382

Created derived feature: SLG
Train - SLG range: 0.274 to 0.491
Test - SLG range: 0.261 to 0.488

Created derived feature: OPS
Train - OPS range: 0.539 to 0.870
Test - OPS range: 0.530 to 0.870

Cre

In [4]:
# Select only the default features from DATA_DESCRIPTION.md
# default_features = [
#     # Basic Statistics
#     'G', 'HR', 'SHO', 'SV', 'IPouts', 'FP', 'ERA', 'ER', 'E',

#     # Derived Features
#     'Expected_Wins', 'Times_On_Base', 'BB_Rate', 'HR_Rate', 'OPS', 'Times_On_Base_Allowed', 
#     'WHIP', 'K_per_9', 'HR_per_9', 'mlb_rpg',
    
#     # Era Indicators
#     'era_1', 'era_2', 'era_3', 'era_4', 'era_5', 'era_6', 'era_7', 'era_8',
    
#     # Decade Indicators
#     'decade_1910', 'decade_1920', 'decade_1930', 'decade_1940', 'decade_1950',
#     'decade_1960', 'decade_1970', 'decade_1980', 'decade_1990', 'decade_2000', 'decade_2010'
# ]

# default_features = [
#     # Basic Statistics
#     'G', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF',
#     'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA',
#     'E', 'DP', 'FP', 'attendance', 'BPF', 'PPF',
    
#     # Derived Features
#     'Expected_Wins', 'Times_On_Base', 'Times_On_Base_Allowed', 'mlb_rpg',

#     # 'Era_Adjusted_OBP', 'Era_Adjusted_SLG', 'Era_Adjusted_OPS', 'Era_Adjusted_WHIP',
#     # 'Era_Adjusted_K_per_9', 'Era_Adjusted_HR_per_9', 'Era_Adjusted_BB_Rate', 'Era_Adjusted_HR_Rate',
#     'BB_Rate', 'HR_Rate', 'OBP', 'SLG', 'OPS', 'WHIP', 'K_per_9', 'HR_per_9',
    
#     # 'PEI', 'REI',
    
#     # # Era Indicators
#     # 'era_1', 'era_2', 'era_3', 'era_4', 'era_5', 'era_6', 'era_7', 'era_8',
    
#     # Decade Indicators
#     'decade_1910', 'decade_1920', 'decade_1930', 'decade_1940', 'decade_1950',
#     'decade_1960', 'decade_1970', 'decade_1980', 'decade_1990', 'decade_2000', 'decade_2010'
#  ]

default_features = [
    # Basic Statistics
    'G', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF',
    'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA',
    'E', 'DP', 'FP', 'attendance', 'BPF', 'PPF',
    
    # Derived Features
    'Expected_Wins', 'Times_On_Base', 'Times_On_Base_Allowed', 'mlb_rpg',

    'Era_Adjusted_OBP', 'Era_Adjusted_SLG', 'Era_Adjusted_OPS', 'Era_Adjusted_WHIP',
    'Era_Adjusted_K_per_9', 'Era_Adjusted_HR_per_9', 'Era_Adjusted_BB_Rate', 'Era_Adjusted_HR_Rate',
    
    'OBP', 'SLG', 'OPS', 'WHIP', 'K_per_9', 'HR_per_9', 'BB_Rate', 'HR_Rate', 
    
    'PEI', 'REI',
    
    # Era Indicators
    'era_1', 'era_2', 'era_3', 'era_4', 'era_5', 'era_6', 'era_7', 'era_8',
    
    # Decade Indicators
    'decade_1910', 'decade_1920', 'decade_1930', 'decade_1940', 'decade_1950',
    'decade_1960', 'decade_1970', 'decade_1980', 'decade_1990', 'decade_2000', 'decade_2010'
 ]

# Filter features that exist in both training data AND test data
available_features = [col for col in default_features 
                     if col in train_df.columns and col in test_df.columns]
print(f"Number of available default features: {len(available_features)}")

# Print available features in a column
print("Available features:")
for feature in available_features:
    print(feature)


Number of available default features: 65
Available features:
G
R
AB
H
2B
3B
HR
BB
SO
SB
RA
ER
ERA
CG
SHO
SV
IPouts
HA
HRA
BBA
SOA
E
DP
FP
Expected_Wins
Times_On_Base
Times_On_Base_Allowed
mlb_rpg
Era_Adjusted_OBP
Era_Adjusted_SLG
Era_Adjusted_OPS
Era_Adjusted_WHIP
Era_Adjusted_K_per_9
Era_Adjusted_HR_per_9
Era_Adjusted_BB_Rate
Era_Adjusted_HR_Rate
OBP
SLG
OPS
WHIP
K_per_9
HR_per_9
BB_Rate
HR_Rate
PEI
REI
era_1
era_2
era_3
era_4
era_5
era_6
era_7
era_8
decade_1910
decade_1920
decade_1930
decade_1940
decade_1950
decade_1960
decade_1970
decade_1980
decade_1990
decade_2000
decade_2010


In [5]:
# Prepare training data (split the train.csv for model evaluation)
X_full = train_df[available_features]
y_full = train_df['W']

# Split training data into train/validation sets for model evaluation
X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42
)

# Prepare final test data for predictions (this has no target variable)
X_test_final = test_df[available_features]

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Final test set shape: {X_test_final.shape}")

Training set shape: (1449, 65)
Validation set shape: (363, 65)
Final test set shape: (453, 65)


In [6]:
# Remove highly correlated features
import pandas as pd
import numpy as np

def remove_correlated_features(X_train, X_test, threshold=0.95, verbose=True):
    """
    Remove highly correlated features from training and test sets.
    
    Parameters:
    - X_train: Training feature DataFrame
    - X_test: Test feature DataFrame  
    - threshold: Correlation threshold (default 0.95)
    - verbose: Print information about removed features
    
    Returns:
    - X_train_filtered, X_test_filtered: DataFrames with correlated features removed
    """
    
    # Calculate correlation matrix
    corr_matrix = X_train.corr().abs()
    
    # Find pairs of highly correlated features
    upper_tri = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    
    # Find features to remove (those with correlation > threshold)
    features_to_remove = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    
    if verbose:
        print(f"🔍 CORRELATION ANALYSIS")
        print(f"{'='*50}")
        print(f"Correlation threshold: {threshold}")
        print(f"Original features: {X_train.shape[1]}")
        print(f"Features to remove: {len(features_to_remove)}")
        
        if features_to_remove:
            print(f"\nHighly correlated features to remove:")
            for feature in features_to_remove:
                # Find what it's correlated with
                high_corr = upper_tri[feature].dropna()
                high_corr = high_corr[high_corr > threshold]
                if len(high_corr) > 0:
                    corr_with = high_corr.index[0]
                    corr_value = high_corr.iloc[0]
                    print(f"  • {feature} (corr={corr_value:.3f} with {corr_with})")
        else:
            print(f"\n✅ No highly correlated features found above threshold {threshold}")
    
    # Remove highly correlated features from both datasets
    X_train_filtered = X_train.drop(columns=features_to_remove)
    X_test_filtered = X_test.drop(columns=features_to_remove)
    
    if verbose:
        print(f"\nFeatures after removal: {X_train_filtered.shape[1]}")
        print(f"Features removed: {len(features_to_remove)}")
        if len(features_to_remove) > 0:
            improvement = len(features_to_remove) / X_train.shape[1] * 100
            print(f"Dimensionality reduction: {improvement:.1f}%")
    
    return X_train_filtered, X_test_filtered

# Apply correlation removal to our datasets
# Store original datasets for backup
X_full_original = X_full.copy()
X_test_final_original = X_test_final.copy()

# Remove correlated features
X_full_filtered, X_test_final_filtered = remove_correlated_features(
    X_full, X_test_final, 
    threshold=0.95, 
    verbose=True
)

# Update the main datasets (so later cells use the filtered versions)
X_full = X_full_filtered
X_test_final = X_test_final_filtered

# Update available_features list to match the filtered features
available_features_filtered = list(X_full.columns)

print(f"\n📊 UPDATED DATASET INFO")
print(f"{'='*50}")
print(f"X_full shape: {X_full.shape}")
print(f"X_test_final shape: {X_test_final.shape}")
print(f"Available features updated: {len(available_features_filtered)}")

# Verify both datasets have the same features
assert list(X_full.columns) == list(X_test_final.columns), "Feature mismatch between train and test!"
print(f"✅ Feature alignment verified between train and test sets")

# Update available_features for downstream compatibility
available_features = available_features_filtered

print(f"\n🔄 Variables updated for downstream compatibility:")
print(f"  • X_full: {X_full.shape}")
print(f"  • X_test_final: {X_test_final.shape}")  
print(f"  • available_features: {len(available_features)} features")
print(f"\n💡 To disable correlation removal, simply comment out this entire cell")

🔍 CORRELATION ANALYSIS
Correlation threshold: 0.95
Original features: 65
Features to remove: 13

Highly correlated features to remove:
  • ERA (corr=0.959 with RA)
  • FP (corr=0.996 with E)
  • Era_Adjusted_K_per_9 (corr=0.953 with SOA)
  • Era_Adjusted_HR_per_9 (corr=0.981 with HRA)
  • Era_Adjusted_HR_Rate (corr=0.979 with HR)
  • OPS (corr=0.969 with SLG)
  • K_per_9 (corr=0.999 with SOA)
  • HR_per_9 (corr=0.999 with HRA)
  • BB_Rate (corr=0.982 with BB)
  • HR_Rate (corr=0.999 with HR)
  • PEI (corr=0.959 with Era_Adjusted_HR_Rate)
  • decade_1910 (corr=1.000 with era_1)
  • decade_2010 (corr=1.000 with era_8)

Features after removal: 52
Features removed: 13
Dimensionality reduction: 20.0%

📊 UPDATED DATASET INFO
X_full shape: (1812, 52)
X_test_final shape: (453, 52)
Available features updated: 52
✅ Feature alignment verified between train and test sets

🔄 Variables updated for downstream compatibility:
  • X_full: (1812, 52)
  • X_test_final: (453, 52)
  • available_features: 52

In [7]:
# Import boosting libraries and Optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_validate, cross_val_score
import time
import warnings
import optuna
from optuna.samplers import TPESampler

# Silence warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("OPTUNA-OPTIMIZED BOOSTING MODELS COMPARISON")
print("="*60)

# Prepare data
X = X_full
y = y_full

print(f"\nDataset shape: {X.shape}")
print(f"Features being used: {list(X.columns)}")

# Define objective functions for Optuna hyperparameter optimization
def xgboost_objective(trial):
    """Objective function for XGBoost hyperparameter tuning"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'random_state': 42,
        'verbosity': 0,
        'tree_method': 'hist'
    }
    
    # Try GPU first, fallback to CPU if needed
    try:
        params['device'] = 'cuda'
        model = XGBRegressor(**params)
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error', n_jobs=1)
    except:
        params['device'] = 'cpu'
        model = XGBRegressor(**params)
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    
    return -scores.mean()  # Optuna minimizes, so negate MAE

def catboost_objective(trial):
    """Objective function for CatBoost hyperparameter tuning"""
    params = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'depth': trial.suggest_int('depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_seed': 42,
        'verbose': False
    }
    
    # Try GPU first, fallback to CPU if needed
    try:
        params['task_type'] = 'GPU'
        model = CatBoostRegressor(**params)
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error', n_jobs=1)
    except:
        params['task_type'] = 'CPU'
        model = CatBoostRegressor(**params)
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    
    return -scores.mean()

# Optimize hyperparameters for each model
print("\n🔍 HYPERPARAMETER OPTIMIZATION")
print("-" * 50)

optimized_params = {}
optimization_results = {}

# XGBoost optimization
print("\nOptimizing XGBoost hyperparameters...")
start_time = time.time()
xgb_study = optuna.create_study(
    direction='minimize',
    sampler=TPESampler(seed=42)
)
xgb_study.optimize(xgboost_objective, n_trials=50, show_progress_bar=False)
xgb_time = time.time() - start_time

optimized_params['XGBoost'] = xgb_study.best_params
optimization_results['XGBoost'] = {
    'best_mae': xgb_study.best_value,
    'optimization_time': xgb_time,
    'n_trials': len(xgb_study.trials)
}
print(f"  Best MAE: {xgb_study.best_value:.4f} (Time: {xgb_time:.1f}s)")

# CatBoost optimization
print("\nOptimizing CatBoost hyperparameters...")
start_time = time.time()
cat_study = optuna.create_study(
    direction='minimize',
    sampler=TPESampler(seed=42)
)
cat_study.optimize(catboost_objective, n_trials=50, show_progress_bar=False)
cat_time = time.time() - start_time

optimized_params['CatBoost'] = cat_study.best_params
optimization_results['CatBoost'] = {
    'best_mae': cat_study.best_value,
    'optimization_time': cat_time,
    'n_trials': len(cat_study.trials)
}
print(f"  Best MAE: {cat_study.best_value:.4f} (Time: {cat_time:.1f}s)")

print(f"\n📋 OPTIMIZATION SUMMARY")
print("-" * 50)
for model_name, result in optimization_results.items():
    print(f"{model_name}:")
    print(f"  Best CV MAE: {result['best_mae']:.4f}")
    print(f"  Optimization time: {result['optimization_time']:.1f}s")
    print(f"  Trials completed: {result['n_trials']}")
    print()

OPTUNA-OPTIMIZED BOOSTING MODELS COMPARISON

Dataset shape: (1812, 52)
Features being used: ['G', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'RA', 'ER', 'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'Expected_Wins', 'Times_On_Base', 'Times_On_Base_Allowed', 'mlb_rpg', 'Era_Adjusted_OBP', 'Era_Adjusted_SLG', 'Era_Adjusted_OPS', 'Era_Adjusted_WHIP', 'Era_Adjusted_BB_Rate', 'OBP', 'SLG', 'WHIP', 'REI', 'era_1', 'era_2', 'era_3', 'era_4', 'era_5', 'era_6', 'era_7', 'era_8', 'decade_1920', 'decade_1930', 'decade_1940', 'decade_1950', 'decade_1960', 'decade_1970', 'decade_1980', 'decade_1990', 'decade_2000']

🔍 HYPERPARAMETER OPTIMIZATION
--------------------------------------------------

Optimizing XGBoost hyperparameters...
  Best MAE: 3.0262 (Time: 31.8s)

Optimizing CatBoost hyperparameters...
  Best MAE: 3.0262 (Time: 31.8s)

Optimizing CatBoost hyperparameters...
  Best MAE: 3.0056 (Time: 183.1s)

📋 OPTIMIZATION SUMMARY
-----------------------------------

In [8]:
# Build models with optimized parameters and perform detailed comparison
print("\n🏗️ BUILDING OPTIMIZED MODELS")
print("-" * 50)

# Create models with optimized parameters
def create_optimized_model(model_name, params):
    """Create a model instance with optimized parameters"""
    if model_name == 'XGBoost':
        # Try GPU first, fallback to CPU
        try:
            params_gpu = params.copy()
            params_gpu['device'] = 'cuda'
            params_gpu['tree_method'] = 'hist'
            params_gpu['verbosity'] = 0
            model = XGBRegressor(**params_gpu)
            # Test if GPU works
            model.fit(X[:100], y[:100])
            return model, '✅ GPU'
        except:
            params_cpu = params.copy()
            params_cpu['device'] = 'cpu'
            params_cpu['tree_method'] = 'hist' 
            params_cpu['verbosity'] = 0
            return XGBRegressor(**params_cpu), '⚠️ CPU'
            
    elif model_name == 'CatBoost':
        try:
            params_gpu = params.copy()
            params_gpu['task_type'] = 'GPU'
            params_gpu['verbose'] = False
            model = CatBoostRegressor(**params_gpu)
            # Test if GPU works
            model.fit(X[:100], y[:100])
            return model, '✅ GPU'
        except:
            params_cpu = params.copy()
            params_cpu['task_type'] = 'CPU'
            params_cpu['verbose'] = False
            return CatBoostRegressor(**params_cpu), '⚠️ CPU'

# Build optimized models
optimized_models = {}
cv_results_optimized = {}

for name, params in optimized_params.items():
    print(f"\nBuilding optimized {name}...")
    start_time = time.time()
    
    model, gpu_status = create_optimized_model(name, params)
    
    # Perform cross-validation
    cv_scores = cross_validate(
        model, X, y,
        cv=5,
        scoring=['r2', 'neg_mean_absolute_error'],
        return_train_score=True,
        n_jobs=1 if 'GPU' in gpu_status else -1
    )
    
    end_time = time.time()
    
    cv_results_optimized[name] = {
        'test_r2': cv_scores['test_r2'].mean(),
        'test_r2_std': cv_scores['test_r2'].std(),
        'test_mae': -cv_scores['test_neg_mean_absolute_error'].mean(),
        'test_mae_std': cv_scores['test_neg_mean_absolute_error'].std(),
        'train_r2': cv_scores['train_r2'].mean(),
        'overfitting': cv_scores['train_r2'].mean() - cv_scores['test_r2'].mean(),
        'time': end_time - start_time,
        'gpu_status': gpu_status
    }
    
    optimized_models[name] = model
    print(f"  {gpu_status} | CV MAE: {cv_results_optimized[name]['test_mae']:.4f}")

print("\n" + "="*90)
print("OPTIMIZED MODELS RESULTS SUMMARY")
print("="*90)
print(f"{'Model':<22} {'Test R²':<10} {'Test MAE':<11} {'Overfitting':<13} {'Time (s)':<10} {'GPU':<10}")
print("-" * 90)

# Sort by Test MAE (lower is better) for initial ranking
sorted_results = sorted(cv_results_optimized.items(), key=lambda x: x[1]['test_mae'])

for name, result in sorted_results:
    overfit_warning = "⚠️" if result['overfitting'] > 0.05 else "✓"
    gpu_icon = "🚀" if "GPU" in result['gpu_status'] else "💻"
    print(f"{name:<22} {result['test_r2']:.4f}    {result['test_mae']:.4f}     "
          f"{result['overfitting']:>6.4f} {overfit_warning:<5} {result['time']:>6.1f}    {gpu_icon}")

print(f"\n🎯 INTELLIGENT MODEL SELECTION (Balancing Performance & Overfitting)")
print("-" * 70)

# Select best model considering both performance and overfitting
def select_best_model_with_overfitting_control(results, overfitting_threshold=0.05, mae_tolerance=0.01):
    """
    Select the best model balancing performance and overfitting.
    
    Args:
        results: Dictionary of model results
        overfitting_threshold: Maximum acceptable overfitting gap (train_r2 - test_r2)
        mae_tolerance: MAE tolerance for accepting a less overfitting model over the best performer
    
    Returns:
        Tuple of (best_model_name, reason)
    """
    # Sort by MAE first
    sorted_by_mae = sorted(results.items(), key=lambda x: x[1]['test_mae'])
    
    # Find models that don't overfit significantly
    non_overfitting_models = [
        (name, result) for name, result in sorted_by_mae 
        if result['overfitting'] <= overfitting_threshold
    ]
    
    best_mae_model = sorted_by_mae[0]
    best_mae = best_mae_model[1]['test_mae']
    
    if non_overfitting_models:
        # Check if the best non-overfitting model is within acceptable MAE tolerance
        best_non_overfit = non_overfitting_models[0]
        mae_diff = best_non_overfit[1]['test_mae'] - best_mae
        
        if mae_diff <= mae_tolerance:
            return best_non_overfit[0], f"Selected for low overfitting ({best_non_overfit[1]['overfitting']:.4f}) with minimal MAE penalty ({mae_diff:.4f})"
        else:
            # Check if the best MAE model overfits significantly
            if best_mae_model[1]['overfitting'] > overfitting_threshold:
                return best_non_overfit[0], f"Selected to avoid overfitting. Best MAE model overfits by {best_mae_model[1]['overfitting']:.4f}"
            else:
                return best_mae_model[0], f"Selected for best MAE ({best_mae:.4f}) with acceptable overfitting ({best_mae_model[1]['overfitting']:.4f})"
    else:
        # All models overfit, choose the one with least overfitting among top performers
        print("  ⚠️ All models show overfitting. Selecting least overfitting among top 3 MAE performers.")
        top_3_mae = sorted_by_mae[:3]
        least_overfit_of_top3 = min(top_3_mae, key=lambda x: x[1]['overfitting'])
        return least_overfit_of_top3[0], f"Least overfitting ({least_overfit_of_top3[1]['overfitting']:.4f}) among top 3 MAE performers"

# Apply intelligent model selection
best_model_name, selection_reason = select_best_model_with_overfitting_control(cv_results_optimized)
best_model = optimized_models[best_model_name]
best_mae = cv_results_optimized[best_model_name]['test_mae']
best_overfitting = cv_results_optimized[best_model_name]['overfitting']

print(f"\nSelected Model: {best_model_name}")
print(f"Selection Reason: {selection_reason}")
print(f"MAE: {best_mae:.4f}, Overfitting: {best_overfitting:.4f}")

# Show comparison with pure MAE-based selection
pure_mae_best = sorted_results[0][0]
if pure_mae_best != best_model_name:
    pure_mae_result = cv_results_optimized[pure_mae_best]
    print(f"\nComparison with pure MAE selection:")
    print(f"  Pure MAE Best: {pure_mae_best} (MAE: {pure_mae_result['test_mae']:.4f}, Overfitting: {pure_mae_result['overfitting']:.4f})")
    print(f"  Selected Model: {best_model_name} (MAE: {best_mae:.4f}, Overfitting: {best_overfitting:.4f})")
    mae_diff = best_mae - pure_mae_result['test_mae']
    overfit_improvement = pure_mae_result['overfitting'] - best_overfitting
    print(f"  Trade-off: +{mae_diff:.4f} MAE for -{overfit_improvement:.4f} overfitting reduction")

print(f"\n🏆 BEST OPTIMIZED MODEL: {best_model_name}")
print(f"   CV MAE: {best_mae:.4f} (±{cv_results_optimized[best_model_name]['test_mae_std']:.4f})")
print(f"   CV R²: {cv_results_optimized[best_model_name]['test_r2']:.4f} (±{cv_results_optimized[best_model_name]['test_r2_std']:.4f})")
print(f"   Overfitting: {best_overfitting:.4f} ({'⚠️' if best_overfitting > 0.05 else '✓'} {'High' if best_overfitting > 0.05 else 'Acceptable'})")

# Display optimized parameters
print(f"\n🔧 OPTIMIZED PARAMETERS FOR {best_model_name}:")
print("-" * 40)
for param, value in optimized_params[best_model_name].items():
    if isinstance(value, float):
        print(f"  {param}: {value:.4f}")
    else:
        print(f"  {param}: {value}")

# Feature importance for best model
print(f"\n📊 FEATURE IMPORTANCE ({best_model_name})")
print("-" * 40)
print(f"Training {best_model_name} on full dataset for feature importance...")

best_model.fit(X, y)

if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 15 Features:")
    for i, row in importance_df.head(15).iterrows():
        print(f"  {row['feature']:>20}: {row['importance']:.4f}")

print(f"\n✨ OPTIMIZATION COMPLETE!")
print(f"   Best model improved MAE through Optuna hyperparameter tuning")
print(f"   Ready for ensemble or final predictions")


🏗️ BUILDING OPTIMIZED MODELS
--------------------------------------------------

Building optimized XGBoost...
  ✅ GPU | CV MAE: 3.0306

Building optimized CatBoost...
  ✅ GPU | CV MAE: 3.0306

Building optimized CatBoost...
  ✅ GPU | CV MAE: 3.0077

OPTIMIZED MODELS RESULTS SUMMARY
Model                  Test R²    Test MAE    Overfitting   Time (s)   GPU       
------------------------------------------------------------------------------------------
CatBoost               0.9149    3.0077     0.0350 ✓        4.3    🚀
XGBoost                0.9146    3.0306     0.0324 ✓        1.5    🚀

🎯 INTELLIGENT MODEL SELECTION (Balancing Performance & Overfitting)
----------------------------------------------------------------------

Selected Model: CatBoost
Selection Reason: Selected for low overfitting (0.0350) with minimal MAE penalty (0.0000)
MAE: 3.0077, Overfitting: 0.0350

🏆 BEST OPTIMIZED MODEL: CatBoost
   CV MAE: 3.0077 (±0.0861)
   CV R²: 0.9149 (±0.0054)
   Overfitting: 0.0350 (✓ 

In [9]:
# Import linear model libraries and continue using Optuna
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, HuberRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_score, KFold
import time
import warnings
from sklearn.exceptions import ConvergenceWarning

# Comprehensive warning suppression for clean output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="optuna") 
warnings.filterwarnings("ignore", category=ConvergenceWarning)  # 🔧 Suppress all convergence warnings
warnings.filterwarnings("ignore", message="Objective did not converge")
warnings.filterwarnings("ignore", message=".*coordinate_descent.*")  # 🔧 Suppress coordinate descent warnings

print("OPTUNA-OPTIMIZED LINEAR MODELS COMPARISON")
print("="*60)

# Prepare data
X_linear = X_full
y_linear = y_full

print(f"\nDataset shape: {X_linear.shape}")
print(f"Using {len(available_features)} engineered features")

# Define objective functions for Optuna hyperparameter optimization
def ridge_objective(trial):
    """Objective function for Ridge regression hyperparameter tuning"""
    alpha = trial.suggest_float('alpha', 0.01, 100.0, log=True)
    
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('model', Ridge(alpha=alpha, random_state=42))
    ])
    
    scores = cross_val_score(model, X_linear, y_linear, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    return -scores.mean()

def lasso_objective(trial):
    """Objective function for Lasso regression hyperparameter tuning"""
    alpha = trial.suggest_float('alpha', 0.01, 10.0, log=True)  # 🔧 Raised minimum alpha for better convergence
    max_iter = trial.suggest_int('max_iter', 10000, 20000)  # 🔧 Much higher iteration range
    
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('model', Lasso(alpha=alpha, max_iter=max_iter, tol=1e-3, random_state=42, warm_start=False))  # 🔧 Relaxed tolerance, no warm start
    ])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # 🔧 Local warning suppression
        scores = cross_val_score(model, X_linear, y_linear, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    return -scores.mean()

def elasticnet_objective(trial):
    """Objective function for ElasticNet regression hyperparameter tuning"""
    alpha = trial.suggest_float('alpha', 0.01, 10.0, log=True)  # 🔧 Raised minimum alpha
    l1_ratio = trial.suggest_float('l1_ratio', 0.1, 0.9)
    max_iter = trial.suggest_int('max_iter', 10000, 20000)  # 🔧 Much higher iteration range
    
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('model', ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter, tol=1e-3, random_state=42, warm_start=False))  # 🔧 Relaxed tolerance
    ])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # 🔧 Local warning suppression  
        scores = cross_val_score(model, X_linear, y_linear, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    return -scores.mean()

def huber_objective(trial):
    """Objective function for Huber regression hyperparameter tuning"""
    epsilon = trial.suggest_float('epsilon', 1.1, 3.0)
    alpha = trial.suggest_float('alpha', 0.0001, 1.0, log=True)
    max_iter = trial.suggest_int('max_iter', 1000, 5000)
    
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('model', HuberRegressor(epsilon=epsilon, alpha=alpha, max_iter=max_iter, tol=1e-05))
    ])
    
    scores = cross_val_score(model, X_linear, y_linear, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    return -scores.mean()

def polynomial_ridge_objective(trial):
    """Objective function for Polynomial Ridge regression hyperparameter tuning"""
    degree = trial.suggest_int('degree', 2, 3)
    alpha = trial.suggest_float('alpha', 0.1, 1000.0, log=True)
    include_bias = trial.suggest_categorical('include_bias', [True, False])
    
    model = Pipeline([
        ('poly', PolynomialFeatures(degree=degree, include_bias=include_bias)),
        ('scaler', StandardScaler()),
        ('model', Ridge(alpha=alpha, random_state=42))
    ])
    
    scores = cross_val_score(model, X_linear, y_linear, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    return -scores.mean()

# Optimize hyperparameters for each linear model
print("\n🔍 LINEAR MODELS HYPERPARAMETER OPTIMIZATION")
print("-" * 50)

optimized_linear_params = {}
linear_optimization_results = {}

# List of models to optimize
linear_models_to_optimize = [
    ('Ridge', ridge_objective),
    ('Lasso', lasso_objective), 
    ('ElasticNet', elasticnet_objective),
    ('Huber', huber_objective),
    ('Polynomial_Ridge', polynomial_ridge_objective)
]

for model_name, objective_func in linear_models_to_optimize:
    print(f"\nOptimizing {model_name} hyperparameters...")
    start_time = time.time()
    
    # Create study for this model
    study = optuna.create_study(
        direction='minimize',
        sampler=TPESampler(seed=42)
    )
    
    try:
        study.optimize(objective_func, n_trials=30, show_progress_bar=False)
        optimization_time = time.time() - start_time
        
        optimized_linear_params[model_name] = study.best_params
        linear_optimization_results[model_name] = {
            'best_mae': study.best_value,
            'optimization_time': optimization_time,
            'n_trials': len(study.trials),
            'status': 'Success'
        }
        print(f"  ✅ Best MAE: {study.best_value:.4f} (Time: {optimization_time:.1f}s)")
        
    except Exception as e:
        print(f"  ❌ Failed: {str(e)}")
        linear_optimization_results[model_name] = {
            'status': 'Failed',
            'error': str(e)
        }

# Also include basic Linear Regression (no hyperparameters to optimize)
print(f"\nTesting Linear Regression (no hyperparameters)...")
start_time = time.time()
linear_reg_model = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
scores = cross_val_score(linear_reg_model, X_linear, y_linear, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
linear_reg_time = time.time() - start_time

optimized_linear_params['LinearRegression'] = {}
linear_optimization_results['LinearRegression'] = {
    'best_mae': -scores.mean(),
    'optimization_time': linear_reg_time,
    'n_trials': 1,
    'status': 'Success'
}
print(f"  ✅ MAE: {-scores.mean():.4f} (Time: {linear_reg_time:.1f}s)")

print(f"\n📋 LINEAR MODELS OPTIMIZATION SUMMARY")
print("-" * 60)
successful_linear = {k: v for k, v in linear_optimization_results.items() if v.get('status') == 'Success'}
for model_name, result in successful_linear.items():
    print(f"{model_name}:")
    print(f"  Best CV MAE: {result['best_mae']:.4f}")
    print(f"  Optimization time: {result['optimization_time']:.1f}s")
    print(f"  Trials completed: {result['n_trials']}")
    print()

OPTUNA-OPTIMIZED LINEAR MODELS COMPARISON

Dataset shape: (1812, 52)
Using 52 engineered features

🔍 LINEAR MODELS HYPERPARAMETER OPTIMIZATION
--------------------------------------------------

Optimizing Ridge hyperparameters...
  ✅ Best MAE: 2.7194 (Time: 2.5s)

Optimizing Lasso hyperparameters...
  ✅ Best MAE: 2.7194 (Time: 2.5s)

Optimizing Lasso hyperparameters...
  ✅ Best MAE: 2.7166 (Time: 0.6s)

Optimizing ElasticNet hyperparameters...
  ✅ Best MAE: 2.7166 (Time: 0.6s)

Optimizing ElasticNet hyperparameters...
  ✅ Best MAE: 2.7196 (Time: 0.7s)

Optimizing Huber hyperparameters...
  ✅ Best MAE: 2.7196 (Time: 0.7s)

Optimizing Huber hyperparameters...
  ✅ Best MAE: 2.7201 (Time: 2.3s)

Optimizing Polynomial_Ridge hyperparameters...
  ✅ Best MAE: 2.7201 (Time: 2.3s)

Optimizing Polynomial_Ridge hyperparameters...
  ✅ Best MAE: 2.7426 (Time: 13.7s)

Testing Linear Regression (no hyperparameters)...
  ✅ MAE: 2.7312 (Time: 0.0s)

📋 LINEAR MODELS OPTIMIZATION SUMMARY
----------------

In [10]:
# Build optimized linear models and perform detailed comparison
print("\n🏗️ BUILDING OPTIMIZED LINEAR MODELS")
print("-" * 50)

# Create models with optimized parameters
def create_optimized_linear_model(model_name, params):
    """Create a linear model instance with optimized parameters"""
    if model_name == 'Ridge':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('model', Ridge(alpha=params['alpha'], random_state=42))
        ])
    
    elif model_name == 'Lasso':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('model', Lasso(
                alpha=params['alpha'], 
                max_iter=params['max_iter'], 
                tol=1e-4,  # 🔧 Improved tolerance
                random_state=42
            ))
        ])
    
    elif model_name == 'ElasticNet':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('model', ElasticNet(
                alpha=params['alpha'], 
                l1_ratio=params['l1_ratio'], 
                max_iter=params['max_iter'],
                tol=1e-4,  # 🔧 Improved tolerance 
                random_state=42
            ))
        ])
    
    elif model_name == 'Huber':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('model', HuberRegressor(
                epsilon=params['epsilon'],
                alpha=params['alpha'],
                max_iter=params['max_iter'],
                tol=1e-05
            ))
        ])
    
    elif model_name == 'Polynomial_Ridge':
        return Pipeline([
            ('poly', PolynomialFeatures(degree=params['degree'], include_bias=params['include_bias'])),
            ('scaler', StandardScaler()),
            ('model', Ridge(alpha=params['alpha'], random_state=42))
        ])
    
    elif model_name == 'LinearRegression':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('model', LinearRegression())
        ])

# Build optimized linear models
optimized_linear_models = {}
cv_results_linear_optimized = {}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

for name in successful_linear.keys():
    print(f"\nBuilding optimized {name}...")
    start_time = time.time()
    
    model = create_optimized_linear_model(name, optimized_linear_params[name])
    
    # Perform cross-validation
    cv_scores = cross_validate(
        model, X_linear, y_linear,
        cv=cv,
        scoring=['r2', 'neg_mean_absolute_error'],
        return_train_score=True,
        n_jobs=-1
    )
    
    end_time = time.time()
    
    cv_results_linear_optimized[name] = {
        'test_r2': cv_scores['test_r2'].mean(),
        'test_r2_std': cv_scores['test_r2'].std(),
        'test_mae': -cv_scores['test_neg_mean_absolute_error'].mean(),
        'test_mae_std': cv_scores['test_neg_mean_absolute_error'].std(),
        'train_r2': cv_scores['train_r2'].mean(),
        'overfitting': cv_scores['train_r2'].mean() - cv_scores['test_r2'].mean(),
        'time': end_time - start_time
    }
    
    optimized_linear_models[name] = model
    print(f"  ✅ CV MAE: {cv_results_linear_optimized[name]['test_mae']:.4f}")

print("\n" + "="*90)
print("OPTIMIZED LINEAR MODELS RESULTS SUMMARY")
print("="*90)
print(f"{'Model':<22} {'Test R²':<10} {'Test MAE':<11} {'Overfitting':<13} {'Time (s)':<10}")
print("-" * 90)

# Sort by Test MAE (lower is better)
sorted_linear_results = sorted(cv_results_linear_optimized.items(), key=lambda x: x[1]['test_mae'])

for name, result in sorted_linear_results:
    overfit_warning = "⚠️" if result['overfitting'] > 0.05 else "✓"
    print(f"{name:<22} {result['test_r2']:.4f}    {result['test_mae']:.4f}     "
          f"{result['overfitting']:>6.4f} {overfit_warning:<5} {result['time']:>6.1f}")

print(f"\n🎯 INTELLIGENT LINEAR MODEL SELECTION")
print("-" * 50)

# Apply intelligent model selection (similar to boosting models)
best_linear_model_name, linear_selection_reason = select_best_model_with_overfitting_control(
    cv_results_linear_optimized, 
    overfitting_threshold=0.05, 
    mae_tolerance=0.01
)
best_linear_model = optimized_linear_models[best_linear_model_name]
best_linear_mae = cv_results_linear_optimized[best_linear_model_name]['test_mae']
best_linear_overfitting = cv_results_linear_optimized[best_linear_model_name]['overfitting']

print(f"\nSelected Linear Model: {best_linear_model_name}")
print(f"Selection Reason: {linear_selection_reason}")
print(f"MAE: {best_linear_mae:.4f}, Overfitting: {best_linear_overfitting:.4f}")

print(f"\n🏆 BEST OPTIMIZED LINEAR MODEL: {best_linear_model_name}")
print(f"   CV MAE: {best_linear_mae:.4f} (±{cv_results_linear_optimized[best_linear_model_name]['test_mae_std']:.4f})")
print(f"   CV R²: {cv_results_linear_optimized[best_linear_model_name]['test_r2']:.4f} (±{cv_results_linear_optimized[best_linear_model_name]['test_r2_std']:.4f})")
print(f"   Overfitting: {best_linear_overfitting:.4f} ({'⚠️' if best_linear_overfitting > 0.05 else '✓'} {'High' if best_linear_overfitting > 0.05 else 'Acceptable'})")

# Display optimized parameters
if optimized_linear_params[best_linear_model_name]:  # Skip if empty (LinearRegression)
    print(f"\n🔧 OPTIMIZED PARAMETERS FOR {best_linear_model_name}:")
    print("-" * 40)
    for param, value in optimized_linear_params[best_linear_model_name].items():
        if isinstance(value, float):
            print(f"  {param}: {value:.4f}")
        else:
            print(f"  {param}: {value}")

print(f"\n🔄 COMPARISON WITH BOOSTING MODELS")
print("-" * 50)
print(f"Best Boosting Model: {best_model_name} (MAE: {best_mae:.4f})")
print(f"Best Linear Model: {best_linear_model_name} (MAE: {best_linear_mae:.4f})")

if best_linear_mae < best_mae:
    print(f"✅ Linear model outperforms boosting by {best_mae - best_linear_mae:.4f}")
else:
    print(f"⚠️ Boosting model outperforms linear by {best_linear_mae - best_mae:.4f}")

print(f"\n✨ LINEAR OPTIMIZATION COMPLETE!")
print(f"   All linear models optimized with Optuna hyperparameter tuning")
print(f"   Ready for enhanced ensemble with optimized linear models")


🏗️ BUILDING OPTIMIZED LINEAR MODELS
--------------------------------------------------

Building optimized Ridge...
  ✅ CV MAE: 2.7263

Building optimized Lasso...
  ✅ CV MAE: 2.7235

Building optimized ElasticNet...
  ✅ CV MAE: 2.7259

Building optimized Huber...
  ✅ CV MAE: 2.7261

Building optimized Polynomial_Ridge...
  ✅ CV MAE: 2.7444

Building optimized LinearRegression...
  ✅ CV MAE: 2.7448

OPTIMIZED LINEAR MODELS RESULTS SUMMARY
Model                  Test R²    Test MAE    Overfitting   Time (s)  
------------------------------------------------------------------------------------------
Lasso                  0.9312    2.7235     0.0041 ✓        0.0
ElasticNet             0.9311    2.7259     0.0041 ✓        0.0
Huber                  0.9309    2.7261     0.0047 ✓        0.0
Ridge                  0.9310    2.7263     0.0046 ✓        0.0
Polynomial_Ridge       0.9300    2.7444     0.0116 ✓        0.2
LinearRegression       0.9301    2.7448     0.0059 ✓        0.0

🎯 INTELLI

# Key Differences Between Weighted and Stacked Ensemble Approaches

| Aspect | Weighted Ensemble | Stacked Ensemble |
|--------|-------------------|------------------|
| **Meta-learning** | ❌ No | ✅ Yes |
| **Learning Method** | Mathematical optimization | Machine learning model |
| **Validation Strategy** | Cross-validation for evaluation | Cross-validation for meta-training |
| **Weight Determination** | Optuna optimization (scipy.optimize) | Meta-learner learns automatically |
| **Model Complexity** | Medium | High |
| **Training Speed** | Fast | Slower |
| **Interpretability** | ✅ Clear weights | ❌ Black-box meta-model |
| **Overfitting Risk** | Lower | Medium |
| **Feature Engineering** | Uses base predictions only | Can learn complex interactions |
| **Hyperparameter Tuning** | Base models + weights | Base models + meta-learner |
| **Implementation Complexity** | Simpler | More complex |
| **Performance Potential** | Good | Potentially higher |

## Summary

- **Weighted Ensemble**: Combines predictions using mathematically optimized weights
- **Stacked Ensemble**: Uses a meta-learner to learn how to combine base model predictions

# Advanced Ensemble Model Selection - Implementation Summary

## 🎯 **Key Improvements Made**

### **1. Weighted Ensemble Enhancement**
- **Before**: Pure performance-based selection (top 2 linear + top 1 boosting)
- **After**: Balanced selection considering both performance (60%) and diversity (40%)
- **Benefit**: Reduces redundancy, improves ensemble complementarity

### **2. Stacked Ensemble Enhancement**  
- **Before**: Parameter variations for diversity (good approach)
- **After**: Added correlation analysis and automatic removal of highly correlated models (r > 0.95)
- **Benefit**: Ensures base models provide unique information to meta-learner

### **3. Selection Strategy Comparison**

| Strategy | Weighted Ensemble | Stacked Ensemble |
|----------|-------------------|------------------|
| **Previous** | Top performers only | Parameter variations |
| **Enhanced** | Balanced performance+diversity | Correlation filtering + variations |
| **Key Metric** | Balanced score (60%/40% split) | Correlation threshold (r ≤ 0.95) |

## 🔍 **How It Works**

### **Balanced Selection Formula**
```
Balanced Score = 0.6 × (1/MAE) + 0.4 × (1 - avg_correlation)
```
- Higher performance score = Lower MAE
- Higher diversity score = Lower correlation with other models

### **Correlation Filtering**
- Calculates pairwise correlations between all model predictions
- Removes the worse-performing model from highly correlated pairs
- Preserves ensemble diversity for better generalization

## 📊 **Expected Benefits**
1. **Reduced Overfitting**: More diverse models generalize better
2. **Better Ensemble Performance**: Complementary predictions combine more effectively
3. **Automatic Optimization**: No manual model selection needed
4. **Robust Validation**: Correlation analysis ensures real diversity

# Weighted Ensemble Implementation

In [12]:
print("PHASE 1: WEIGHTED ENSEMBLE IMPLEMENTATION (OPTUNA-OPTIMIZED)")
print("="*70)

from scipy.optimize import minimize
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error

# First, let's identify our top performing models from Optuna optimization
print("\n1. ADVANCED MODEL SELECTION FOR WEIGHTED ENSEMBLE")
print("-" * 50)

# Generate all model predictions for balanced selection analysis
all_model_predictions = {}
cv = KFold(n_splits=5, shuffle=True, random_state=42)

print("Generating predictions for all Optuna-optimized models...")

# Add linear model predictions
for name, model in optimized_linear_models.items():
    print(f"  Processing linear model: {name}")
    oof_pred = cross_val_predict(model, X_full, y_full, cv=cv, method='predict')
    all_model_predictions[name] = oof_pred

# Add boosting model predictions  
for name, model in optimized_models.items():
    print(f"  Processing boosting model: {name}")
    oof_pred = cross_val_predict(model, X_full, y_full, cv=cv, method='predict')
    all_model_predictions[name] = oof_pred

def balanced_model_selection(model_predictions, target, max_models=4):
    """Select models balancing performance and diversity"""
    models = list(model_predictions.keys())
    scores = {}
    
    for model in models:
        mae = mean_absolute_error(target, model_predictions[model])
        
        # Calculate average correlation with other models
        correlations = []
        for other_model in models:
            if model != other_model:
                corr = abs(np.corrcoef(model_predictions[model], 
                                     model_predictions[other_model])[0, 1])
                correlations.append(corr)
        
        avg_correlation = np.mean(correlations)
        
        # Balanced score: 60% performance + 40% diversity
        performance_score = 1 / mae  # Higher is better
        diversity_score = 1 - avg_correlation  # Higher is better (lower correlation)
        
        balanced_score = 0.6 * performance_score + 0.4 * diversity_score
        scores[model] = {
            'balanced_score': balanced_score,
            'mae': mae,
            'avg_correlation': avg_correlation,
            'performance_score': performance_score,
            'diversity_score': diversity_score
        }
    
    # Select top models by balanced score
    selected = sorted(scores.keys(), key=lambda x: scores[x]['balanced_score'], reverse=True)[:max_models]
    return selected, scores

# Apply balanced selection strategy
print(f"\nApplying balanced selection strategy (performance + diversity)...")
selected_models, selection_scores = balanced_model_selection(all_model_predictions, y_full, max_models=4)

print(f"\n🎯 BALANCED MODEL SELECTION RESULTS")
print("-" * 50)
print(f"Selected {len(selected_models)} models using balanced strategy:")
for i, model in enumerate(selected_models):
    score_info = selection_scores[model]
    print(f"  {i+1}. {model}:")
    print(f"     MAE: {score_info['mae']:.4f}")
    print(f"     Avg Correlation: {score_info['avg_correlation']:.3f}")
    print(f"     Diversity Score: {score_info['diversity_score']:.3f}")
    print(f"     Performance Score: {score_info['performance_score']:.1f}")
    print(f"     Balanced Score: {score_info['balanced_score']:.3f}")

# Build ensemble_models dict with selected models
ensemble_models = {}
for name in selected_models:
    if name in optimized_linear_models:
        ensemble_models[name] = optimized_linear_models[name]
    else:
        ensemble_models[name] = optimized_models[name]

print(f"\n2. ENSEMBLE COMPOSITION (BALANCED SELECTION)")
print("-" * 50)
print(f"Selected {len(ensemble_models)} balanced models for ensemble:")
for name in ensemble_models.keys():
    print(f"  ✅ {name} (Optuna-optimized)")
    
print(f"\nTotal ensemble models: {len(ensemble_models)}")

# Store performance metrics for weight calculation using Optuna results
model_performance = {}
for name in ensemble_models.keys():
    if name in cv_results_linear_optimized:  # Linear model
        model_performance[name] = {
            'mae': cv_results_linear_optimized[name]['test_mae'],
            'r2': cv_results_linear_optimized[name]['test_r2']
        }
    else:  # Boosting model
        model_performance[name] = {
            'mae': cv_results_optimized[name]['test_mae'], 
            'r2': cv_results_optimized[name]['test_r2']
        }

print(f"\n3. SELECTED MODEL PERFORMANCE SUMMARY")
print("-" * 50)
for name, perf in model_performance.items():
    print(f"{name}: MAE = {perf['mae']:.4f}, R² = {perf['r2']:.4f}")

# Display the Optuna-optimized parameters being used
print(f"\n4. OPTUNA PARAMETERS IN USE")
print("-" * 50)
for name in ensemble_models.keys():
    if name in optimized_linear_params and optimized_linear_params[name]:
        print(f"\n{name} (Linear):")
        for param, value in optimized_linear_params[name].items():
            if isinstance(value, float):
                print(f"  {param}: {value:.4f}")
            else:
                print(f"  {param}: {value}")
    elif name in optimized_params:
        print(f"\n{name} (Boosting):")
        for param, value in optimized_params[name].items():
            if isinstance(value, float) and 'rate' in param:
                print(f"  {param}: {value:.4f}")
            elif isinstance(value, (int, bool, str)):
                print(f"  {param}: {value}")

print(f"\n🚀 Using balanced Optuna-optimized models for superior ensemble diversity!")

PHASE 1: WEIGHTED ENSEMBLE IMPLEMENTATION (OPTUNA-OPTIMIZED)

1. ADVANCED MODEL SELECTION FOR WEIGHTED ENSEMBLE
--------------------------------------------------
Generating predictions for all Optuna-optimized models...
  Processing linear model: Ridge
  Processing linear model: Lasso
  Processing linear model: ElasticNet
  Processing linear model: Huber
  Processing linear model: Polynomial_Ridge
  Processing linear model: Huber
  Processing linear model: Polynomial_Ridge
  Processing linear model: LinearRegression
  Processing boosting model: XGBoost
  Processing linear model: LinearRegression
  Processing boosting model: XGBoost
  Processing boosting model: CatBoost
  Processing boosting model: CatBoost

Applying balanced selection strategy (performance + diversity)...

🎯 BALANCED MODEL SELECTION RESULTS
--------------------------------------------------
Selected 4 models using balanced strategy:
  1. Lasso:
     MAE: 2.7235
     Avg Correlation: 0.997
     Diversity Score: 0.003
 

In [13]:
# Generate out-of-fold predictions for weight optimization with Optuna-optimized models
print("\n5. GENERATING OUT-OF-FOLD PREDICTIONS (OPTUNA MODELS)")
print("-" * 60)

from sklearn.model_selection import cross_val_predict

# Generate OOF predictions for each Optuna-optimized model
oof_predictions = {}
model_names = list(ensemble_models.keys())

# Use the same CV strategy as the original optimization
cv = KFold(n_splits=5, shuffle=True, random_state=42)

for name, model in ensemble_models.items():
    print(f"Generating OOF predictions for {name} (Optuna-optimized)...")
    
    # Use the same CV strategy as before
    oof_pred = cross_val_predict(model, X_full, y_full, cv=cv, method='predict')
    oof_predictions[name] = oof_pred
    
    # Calculate OOF MAE
    oof_mae = mean_absolute_error(y_full, oof_pred)
    print(f"  OOF MAE: {oof_mae:.4f}")

# Create OOF prediction matrix
oof_matrix = np.column_stack([oof_predictions[name] for name in model_names])
print(f"\nOOF prediction matrix shape: {oof_matrix.shape}")

print("\n6. OPTIMIZING ENSEMBLE WEIGHTS FOR OPTUNA MODELS")
print("-" * 60)

def ensemble_mae_objective(weights, predictions, targets):
    """Objective function to minimize: weighted ensemble MAE"""
    weights = np.array(weights)
    weights = weights / weights.sum()  # Normalize to sum to 1
    ensemble_pred = np.dot(predictions, weights)
    return mean_absolute_error(targets, ensemble_pred)

# Initial weights based on inverse MAE (better models get higher weights)
initial_weights = []
for name in model_names:
    mae = model_performance[name]['mae']
    # Inverse weight: lower MAE = higher weight
    weight = 1.0 / mae if mae > 0 else 1.0
    initial_weights.append(weight)

# Normalize initial weights
initial_weights = np.array(initial_weights)
initial_weights = initial_weights / initial_weights.sum()

print("Initial weights (based on Optuna-optimized model performance):")
for i, name in enumerate(model_names):
    print(f"  {name}: {initial_weights[i]:.3f}")

# Constraint: weights must sum to 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0})

# Bounds: each weight between 0 and 1
bounds = [(0.0, 1.0) for _ in range(len(model_names))]

# Optimize weights
result = minimize(
    ensemble_mae_objective,
    initial_weights,
    args=(oof_matrix, y_full),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

optimal_weights = result.x
optimal_mae = result.fun

print(f"\nOptimization successful: {result.success}")
print(f"Optimal Optuna ensemble OOF MAE: {optimal_mae:.4f}")
print("\nOptimal weights for Optuna-optimized models:")
for i, name in enumerate(model_names):
    print(f"  {name}: {optimal_weights[i]:.3f}")

# Calculate improvement over best individual Optuna-optimized model
best_individual_mae = min([model_performance[name]['mae'] for name in model_names])
improvement = best_individual_mae - optimal_mae
print(f"\nImprovement over best individual Optuna model:")
print(f"  Best individual MAE: {best_individual_mae:.4f}")
print(f"  Ensemble MAE: {optimal_mae:.4f}")
print(f"  Improvement: {improvement:.4f} ({improvement/best_individual_mae*100:.2f}%)")

print(f"\n✨ Ensemble optimization complete using Optuna-optimized base models!")


5. GENERATING OUT-OF-FOLD PREDICTIONS (OPTUNA MODELS)
------------------------------------------------------------
Generating OOF predictions for Lasso (Optuna-optimized)...
  OOF MAE: 2.7235
Generating OOF predictions for Huber (Optuna-optimized)...
  OOF MAE: 2.7262
Generating OOF predictions for Ridge (Optuna-optimized)...
  OOF MAE: 2.7264
Generating OOF predictions for ElasticNet (Optuna-optimized)...
  OOF MAE: 2.7260

OOF prediction matrix shape: (1812, 4)

6. OPTIMIZING ENSEMBLE WEIGHTS FOR OPTUNA MODELS
------------------------------------------------------------
Initial weights (based on Optuna-optimized model performance):
  Lasso: 0.250
  Huber: 0.250
  Ridge: 0.250
  ElasticNet: 0.250

Optimization successful: True
Optimal Optuna ensemble OOF MAE: 2.7234

Optimal weights for Optuna-optimized models:
  Lasso: 0.886
  Huber: 0.062
  Ridge: 0.052
  ElasticNet: 0.000

Improvement over best individual Optuna model:
  Best individual MAE: 2.7235
  Ensemble MAE: 2.7234
  Improve

In [14]:
# Train final Optuna-optimized models and generate test predictions
print("\n7. TRAINING FINAL OPTUNA-OPTIMIZED ENSEMBLE MODELS")
print("-" * 60)

# Train each Optuna-optimized model on the full training dataset
final_models = {}
test_predictions = {}

for name, model in ensemble_models.items():
    print(f"Training {name} (Optuna-optimized) on full dataset...")
    
    # Clone and train the Optuna-optimized model
    final_model = model  # Already configured with Optuna parameters
    final_model.fit(X_full, y_full)
    final_models[name] = final_model
    
    # Generate test predictions
    test_pred = final_model.predict(X_test_final)
    test_predictions[name] = test_pred
    
    print(f"  Test predictions range: {test_pred.min():.2f} to {test_pred.max():.2f}")

print(f"\nAll {len(final_models)} Optuna-optimized models trained successfully!")

# Create test prediction matrix
test_matrix = np.column_stack([test_predictions[name] for name in model_names])
print(f"Test prediction matrix shape: {test_matrix.shape}")

print("\n8. GENERATING OPTUNA-ENHANCED ENSEMBLE PREDICTIONS")
print("-" * 60)

# Generate weighted ensemble predictions using Optuna-optimized models
ensemble_test_pred = np.dot(test_matrix, optimal_weights)

print(f"Optuna-enhanced ensemble test predictions:")
print(f"  Range: {ensemble_test_pred.min():.2f} to {ensemble_test_pred.max():.2f}")
print(f"  Mean: {ensemble_test_pred.mean():.2f}")
print(f"  Std: {ensemble_test_pred.std():.2f}")

# Compare with individual Optuna-optimized model predictions
print(f"\nComparison with individual Optuna-optimized models:")
for i, name in enumerate(model_names):
    individual_pred = test_predictions[name]
    weight = optimal_weights[i]
    print(f"  {name} (weight={weight:.3f}): mean={individual_pred.mean():.2f}, std={individual_pred.std():.2f}")

print(f"\n9. CREATING OPTUNA-ENHANCED SUBMISSION FILE")
print("-" * 60)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'ID': test_df['ID'],  # Use the actual ID column from test.csv
    'W': ensemble_test_pred
})

# Generate timestamp for unique filename
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
submission_filename = f"submission_optuna_weighted_ensemble_{timestamp}.csv"
submission_path = SUB_DIR / submission_filename

# Save submission
submission_df.to_csv(submission_path, index=False)

print(f"✅ Optuna-enhanced submission saved: {submission_filename}")
print(f"📁 Path: {submission_path}")
print(f"📊 Predictions shape: {submission_df.shape}")

# Display first few predictions
print(f"\nFirst 10 predictions:")
print(submission_df.head(10))

print(f"\n10. OPTUNA-ENHANCED WEIGHTED ENSEMBLE SUMMARY")
print("-" * 60)
print(f"Ensemble Composition (Optuna-optimized models):")
for i, name in enumerate(model_names):
    print(f"  {name}: {optimal_weights[i]:.1%}")
print(f"\nExpected Performance:")
print(f"  Cross-validation MAE: {optimal_mae:.4f}")
print(f"  Expected Kaggle score: ~{optimal_mae:.2f}")
print(f"  Improvement vs best individual Optuna model: {improvement:.4f}")
print(f"\n🚀 Phase 1 complete with Optuna-optimized models!")


7. TRAINING FINAL OPTUNA-OPTIMIZED ENSEMBLE MODELS
------------------------------------------------------------
Training Lasso (Optuna-optimized) on full dataset...
  Test predictions range: 44.85 to 109.50
Training Huber (Optuna-optimized) on full dataset...
  Test predictions range: 44.53 to 109.69
Training Ridge (Optuna-optimized) on full dataset...
  Test predictions range: 44.57 to 109.50
Training ElasticNet (Optuna-optimized) on full dataset...
  Test predictions range: 44.79 to 109.02

All 4 Optuna-optimized models trained successfully!
Test prediction matrix shape: (453, 4)

8. GENERATING OPTUNA-ENHANCED ENSEMBLE PREDICTIONS
------------------------------------------------------------
Optuna-enhanced ensemble test predictions:
  Range: 44.81 to 109.51
  Mean: 79.08
  Std: 12.05

Comparison with individual Optuna-optimized models:
  Lasso (weight=0.886): mean=79.08, std=12.05
  Huber (weight=0.062): mean=79.14, std=12.07
  Ridge (weight=0.052): mean=79.13, std=12.05
  ElasticNe

# Stacked Ensemble Implementation

In [15]:
print("STACKED ENSEMBLE IMPLEMENTATION (OPTUNA-OPTIMIZED)")
print("="*60)

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
import warnings
warnings.filterwarnings("ignore")

print(f"\n1. CREATING DIVERSE OPTUNA-OPTIMIZED BASE MODELS") 
print("-" * 60)

# Create diverse base models using Optuna-optimized parameters for better generalization
def create_optuna_linear_model(model_type, params, suffix=""):
    """Create linear model with Optuna-optimized parameters"""
    if model_type == 'Ridge':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('model', Ridge(alpha=params.get('alpha', 1.0), random_state=42))
        ])
    elif model_type == 'Lasso':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('model', Lasso(
                alpha=params.get('alpha', 0.01), 
                max_iter=params.get('max_iter', 10000),
                tol=1e-3,
                random_state=42
            ))
        ])
    elif model_type == 'ElasticNet':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('model', ElasticNet(
                alpha=params.get('alpha', 0.1),
                l1_ratio=params.get('l1_ratio', 0.5),
                max_iter=params.get('max_iter', 10000),
                tol=1e-3,
                random_state=42
            ))
        ])

# Build stacking models using Optuna-optimized parameters
stacking_models = {}

# Linear models with Optuna-optimized parameters + variations for diversity
if 'Ridge' in optimized_linear_params:
    ridge_params = optimized_linear_params['Ridge']
    # Use optimized alpha and create variations
    base_alpha = ridge_params.get('alpha', 1.0)
    stacking_models['Ridge_optuna'] = create_optuna_linear_model('Ridge', ridge_params)
    stacking_models['Ridge_conservative'] = create_optuna_linear_model('Ridge', {'alpha': base_alpha * 5})
    stacking_models['Ridge_aggressive'] = create_optuna_linear_model('Ridge', {'alpha': base_alpha * 0.2})

if 'Lasso' in optimized_linear_params:
    lasso_params = optimized_linear_params['Lasso']
    stacking_models['Lasso_optuna'] = create_optuna_linear_model('Lasso', lasso_params)
    # Create variation
    base_alpha = lasso_params.get('alpha', 0.01)
    stacking_models['Lasso_variation'] = create_optuna_linear_model('Lasso', {
        'alpha': base_alpha * 2,
        'max_iter': lasso_params.get('max_iter', 10000)
    })

if 'ElasticNet' in optimized_linear_params:
    elasticnet_params = optimized_linear_params['ElasticNet']
    stacking_models['ElasticNet_optuna'] = create_optuna_linear_model('ElasticNet', elasticnet_params)

# Tree-based models with Optuna-optimized parameters
if 'XGBoost' in optimized_params:
    xgb_params = optimized_params['XGBoost'].copy()
    # Use Optuna parameters but make conservative for stacking
    xgb_params['n_estimators'] = min(xgb_params.get('n_estimators', 150), 150)  # Cap for speed
    xgb_params['verbosity'] = 0
    xgb_params['random_state'] = 42
    
    # Create XGBoost with Optuna parameters
    stacking_models['XGBoost_optuna'] = XGBRegressor(**xgb_params)
    
    # Create conservative variation
    conservative_params = xgb_params.copy()
    conservative_params['max_depth'] = max(3, xgb_params.get('max_depth', 6) - 1)  # Shallower
    conservative_params['learning_rate'] = xgb_params.get('learning_rate', 0.1) * 0.8  # Slower
    stacking_models['XGBoost_conservative'] = XGBRegressor(**conservative_params)

if 'CatBoost' in optimized_params:
    cat_params = optimized_params['CatBoost'].copy()
    # Use Optuna parameters but make conservative for stacking
    cat_params['iterations'] = min(cat_params.get('iterations', 150), 150)  # Cap for speed
    cat_params['verbose'] = False
    cat_params['random_seed'] = 42
    
    stacking_models['CatBoost_optuna'] = CatBoostRegressor(**cat_params)

# Add the best individual Optuna-optimized models
stacking_models['Best_Linear'] = best_linear_model  # From Optuna optimization
stacking_models['Best_Boosting'] = best_model       # From Optuna optimization

print(f"Base models for stacking (Optuna-optimized): {len(stacking_models)}")
for name in stacking_models.keys():
    print(f"  ✅ {name}")

print(f"\n2. IMPLEMENTING STACKED ENSEMBLE WITH OPTUNA MODELS")
print("-" * 60)

# Use the same CV folds for all models to ensure consistency  
stacking_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Level 1: Generate out-of-fold predictions from Optuna-optimized base models
print("Generating Level 1 out-of-fold predictions with Optuna-optimized models...")

level1_oof_preds = np.zeros((len(X_full), len(stacking_models)))
level1_test_preds = np.zeros((len(X_test_final), len(stacking_models)))

model_names_stack = list(stacking_models.keys())

for i, (name, model) in enumerate(stacking_models.items()):
    print(f"  Processing {name} (Optuna-enhanced)...")
    
    # Generate OOF predictions
    oof_pred = cross_val_predict(model, X_full, y_full, cv=stacking_cv, method='predict')
    level1_oof_preds[:, i] = oof_pred
    
    # Train on full dataset and predict test set
    model_clone = clone(model)
    model_clone.fit(X_full, y_full)
    test_pred = model_clone.predict(X_test_final)
    level1_test_preds[:, i] = test_pred
    
    # Calculate individual model OOF MAE
    oof_mae = mean_absolute_error(y_full, oof_pred)
    print(f"    OOF MAE: {oof_mae:.4f}")

print(f"\nLevel 1 OOF predictions shape: {level1_oof_preds.shape}")
print(f"Level 1 test predictions shape: {level1_test_preds.shape}")
print(f"🚀 All base models use Optuna-optimized parameters!")

STACKED ENSEMBLE IMPLEMENTATION (OPTUNA-OPTIMIZED)

1. CREATING DIVERSE OPTUNA-OPTIMIZED BASE MODELS
------------------------------------------------------------
Base models for stacking (Optuna-optimized): 11
  ✅ Ridge_optuna
  ✅ Ridge_conservative
  ✅ Ridge_aggressive
  ✅ Lasso_optuna
  ✅ Lasso_variation
  ✅ ElasticNet_optuna
  ✅ XGBoost_optuna
  ✅ XGBoost_conservative
  ✅ CatBoost_optuna
  ✅ Best_Linear
  ✅ Best_Boosting

2. IMPLEMENTING STACKED ENSEMBLE WITH OPTUNA MODELS
------------------------------------------------------------
Generating Level 1 out-of-fold predictions with Optuna-optimized models...
  Processing Ridge_optuna (Optuna-enhanced)...
    OOF MAE: 2.7264
  Processing Ridge_conservative (Optuna-enhanced)...
    OOF MAE: 2.7530
  Processing Ridge_aggressive (Optuna-enhanced)...
    OOF MAE: 2.7301
  Processing Lasso_optuna (Optuna-enhanced)...
    OOF MAE: 2.7237
  Processing Lasso_variation (Optuna-enhanced)...
    OOF MAE: 2.7316
  Processing ElasticNet_optuna (Opt

In [16]:
print(f"\n1.1 STACKING MODEL DIVERSITY ANALYSIS")
print("-" * 50)

# Analyze diversity of stacking models before implementing the ensemble
print("Analyzing correlation between stacking models...")

stacking_predictions = {}
temp_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Generate predictions for correlation analysis
for name, model in stacking_models.items():
    print(f"  Generating predictions for {name}...")
    oof_pred = cross_val_predict(model, X_full, y_full, cv=temp_cv, method='predict')
    stacking_predictions[name] = oof_pred

# Calculate correlation matrix for stacking models
stacking_models_list = list(stacking_predictions.keys())
n_models = len(stacking_models_list)
correlation_matrix = np.zeros((n_models, n_models))

for i, model1 in enumerate(stacking_models_list):
    for j, model2 in enumerate(stacking_models_list):
        correlation_matrix[i, j] = np.corrcoef(stacking_predictions[model1], 
                                             stacking_predictions[model2])[0, 1]

print(f"\nStacking models correlation matrix:")
print("-" * 60)

# Display correlation matrix (truncated for readability)
max_display = min(8, len(stacking_models_list))  # Show max 8 models for space
print(f"{'Model':<25}", end="")
for i in range(max_display):
    model_name = stacking_models_list[i][:10]  # Truncate long names
    print(f"{model_name:<12}", end="")
print()

for i, model1 in enumerate(stacking_models_list):
    if i >= max_display:
        break
    model_name = model1[:24]  # Truncate long names
    print(f"{model_name:<25}", end="")
    for j in range(max_display):
        print(f"{correlation_matrix[i, j]:.3f}       ", end="")
    print()

if len(stacking_models_list) > max_display:
    print(f"... ({len(stacking_models_list) - max_display} more models)")

# Identify highly correlated pairs
high_corr_threshold = 0.95  # Very high threshold for stacking (meta-learner can handle some correlation)
correlated_pairs = []

for i, model1 in enumerate(stacking_models_list):
    for j, model2 in enumerate(stacking_models_list[i+1:], i+1):
        correlation = correlation_matrix[i, j]
        if correlation > high_corr_threshold:
            mae1 = mean_absolute_error(y_full, stacking_predictions[model1])
            mae2 = mean_absolute_error(y_full, stacking_predictions[model2])
            correlated_pairs.append({
                'model1': model1,
                'model2': model2,
                'correlation': correlation,
                'mae1': mae1,
                'mae2': mae2,
                'worse_model': model1 if mae1 > mae2 else model2
            })

if correlated_pairs:
    print(f"\n⚠️  Found {len(correlated_pairs)} highly correlated pairs (r > {high_corr_threshold}):")
    models_to_remove = set()
    
    for pair in correlated_pairs:
        print(f"  {pair['model1']} ↔ {pair['model2']}: r = {pair['correlation']:.3f}")
        print(f"    MAE: {pair['mae1']:.4f} vs {pair['mae2']:.4f}")
        print(f"    Removing: {pair['worse_model']}")
        models_to_remove.add(pair['worse_model'])
    
    # Remove highly correlated models
    for model in models_to_remove:
        print(f"    🗑️  Removing {model}")
        del stacking_models[model]
    
    print(f"\n📊 Stacking models after correlation filtering: {len(stacking_models)}")
else:
    print(f"\n✅ All stacking models have acceptable diversity (r ≤ {high_corr_threshold})")

# Calculate average correlation of remaining models
remaining_models = list(stacking_models.keys())
if len(remaining_models) > 1:
    avg_correlations = []
    for i, model1 in enumerate(remaining_models):
        for j, model2 in enumerate(remaining_models[i+1:], i+1):
            model1_idx = stacking_models_list.index(model1)
            model2_idx = stacking_models_list.index(model2)
            avg_correlations.append(abs(correlation_matrix[model1_idx, model2_idx]))
    
    mean_correlation = np.mean(avg_correlations)
    print(f"Average absolute correlation between remaining models: {mean_correlation:.3f}")
    
    if mean_correlation < 0.3:
        diversity_status = "🟢 Excellent diversity"
    elif mean_correlation < 0.6:
        diversity_status = "🟡 Good diversity"
    else:
        diversity_status = "🟠 Moderate diversity"
    
    print(f"Diversity assessment: {diversity_status}")

print(f"\n📋 Final stacking models for ensemble:")
for i, name in enumerate(stacking_models.keys(), 1):
    mae = mean_absolute_error(y_full, stacking_predictions[name]) if name in stacking_predictions else "N/A"
    print(f"  {i:2d}. {name} (MAE: {mae:.4f})" if mae != "N/A" else f"  {i:2d}. {name}")

print(f"\n🎯 Ready for stacking with {len(stacking_models)} diverse base models!")


1.1 STACKING MODEL DIVERSITY ANALYSIS
--------------------------------------------------
Analyzing correlation between stacking models...
  Generating predictions for Ridge_optuna...
  Generating predictions for Ridge_conservative...
  Generating predictions for Ridge_aggressive...
  Generating predictions for Lasso_optuna...
  Generating predictions for Lasso_variation...
  Generating predictions for ElasticNet_optuna...
  Generating predictions for XGBoost_optuna...
  Generating predictions for XGBoost_conservative...
  Generating predictions for XGBoost_conservative...
  Generating predictions for CatBoost_optuna...
  Generating predictions for CatBoost_optuna...
  Generating predictions for Best_Linear...
  Generating predictions for Best_Boosting...
  Generating predictions for Best_Linear...
  Generating predictions for Best_Boosting...

Stacking models correlation matrix:
------------------------------------------------------------
Model                    Ridge_optu  Ridge_con

In [18]:
print(f"\n2. IMPLEMENTING STACKED ENSEMBLE WITH FILTERED MODELS")
print("-" * 60)

from sklearn.model_selection import cross_val_predict
from sklearn.base import clone
from sklearn.metrics import mean_absolute_error

# Use the same CV folds for all models to ensure consistency  
stacking_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Level 1: Generate out-of-fold predictions from filtered Optuna-optimized base models
print("Generating Level 1 out-of-fold predictions with filtered diverse models...")

level1_oof_preds = np.zeros((len(X_full), len(stacking_models)))
level1_test_preds = np.zeros((len(X_test_final), len(stacking_models)))

model_names_stack = list(stacking_models.keys())

for i, (name, model) in enumerate(stacking_models.items()):
    print(f"  Processing {name} (Optuna-enhanced, diversity-filtered)...")
    
    # Generate OOF predictions
    oof_pred = cross_val_predict(model, X_full, y_full, cv=stacking_cv, method='predict')
    level1_oof_preds[:, i] = oof_pred
    
    # Train on full dataset and predict test set
    model_clone = clone(model)
    model_clone.fit(X_full, y_full)
    test_pred = model_clone.predict(X_test_final)
    level1_test_preds[:, i] = test_pred
    
    # Calculate individual model OOF MAE
    oof_mae = mean_absolute_error(y_full, oof_pred)
    print(f"    OOF MAE: {oof_mae:.4f}")

print(f"\nLevel 1 predictions generated:")
print(f"  OOF matrix shape: {level1_oof_preds.shape}")
print(f"  Test matrix shape: {level1_test_preds.shape}")
print(f"  Using {len(model_names_stack)} diverse base models")

# Calculate correlation of Level 1 predictions to verify diversity
print(f"\nLevel 1 prediction diversity check:")

if len(model_names_stack) < 2:
    print(f"  ⚠️ Only {len(model_names_stack)} model available - no correlation analysis needed")
    avg_level1_correlation = 0.0
    diversity_status = "🟡 Single model - no diversity analysis"
else:
    level1_correlations = np.corrcoef(level1_oof_preds.T)
    
    # Handle case where corrcoef returns a scalar (shouldn't happen with >= 2 models, but safety check)
    if level1_correlations.ndim == 0:
        avg_level1_correlation = float(level1_correlations)
    else:
        upper_tri_indices = np.triu_indices_from(level1_correlations, k=1)
        avg_level1_correlation = np.mean(np.abs(level1_correlations[upper_tri_indices]))
    
    print(f"  Average absolute correlation: {avg_level1_correlation:.3f}")
    
    if avg_level1_correlation < 0.3:
        diversity_status = "🟢 Excellent diversity for stacking"
    elif avg_level1_correlation < 0.6:
        diversity_status = "🟡 Good diversity for stacking"
    else:
        diversity_status = "🟠 Moderate diversity for stacking"

print(f"  Diversity status: {diversity_status}")

print(f"\n✅ Level 1 complete - Ready for meta-learner training!")


2. IMPLEMENTING STACKED ENSEMBLE WITH FILTERED MODELS
------------------------------------------------------------
Generating Level 1 out-of-fold predictions with filtered diverse models...
  Processing Best_Linear (Optuna-enhanced, diversity-filtered)...
    OOF MAE: 2.7235

Level 1 predictions generated:
  OOF matrix shape: (1812, 1)
  Test matrix shape: (453, 1)
  Using 1 diverse base models

Level 1 prediction diversity check:
  ⚠️ Only 1 model available - no correlation analysis needed
  Diversity status: 🟡 Single model - no diversity analysis

✅ Level 1 complete - Ready for meta-learner training!


In [19]:
# Level 2: Train meta-learner with Optuna-enhanced parameters
print(f"\n3. TRAINING LEVEL 2 META-LEARNER (OPTUNA-ENHANCED)")
print("-" * 60)

from sklearn.model_selection import cross_val_score

# Use Optuna-optimized parameters for meta-learners too
meta_learners = {}

# Create meta-learners using Optuna-optimized parameters when available
if 'Ridge' in optimized_linear_params:
    ridge_alpha = optimized_linear_params['Ridge'].get('alpha', 1.0)
    meta_learners['Ridge_meta_optuna'] = Ridge(alpha=ridge_alpha)
    meta_learners['Ridge_meta_conservative'] = Ridge(alpha=ridge_alpha * 10)  # More regularized
    meta_learners['Ridge_meta_aggressive'] = Ridge(alpha=ridge_alpha * 0.1)   # Less regularized

if 'Lasso' in optimized_linear_params:
    lasso_alpha = optimized_linear_params['Lasso'].get('alpha', 0.01)
    lasso_max_iter = optimized_linear_params['Lasso'].get('max_iter', 10000)
    meta_learners['Lasso_meta_optuna'] = Lasso(alpha=lasso_alpha, max_iter=lasso_max_iter, tol=1e-3)

if 'ElasticNet' in optimized_linear_params:
    en_alpha = optimized_linear_params['ElasticNet'].get('alpha', 0.1)
    en_l1_ratio = optimized_linear_params['ElasticNet'].get('l1_ratio', 0.5)
    en_max_iter = optimized_linear_params['ElasticNet'].get('max_iter', 10000)
    meta_learners['ElasticNet_meta_optuna'] = ElasticNet(
        alpha=en_alpha, l1_ratio=en_l1_ratio, max_iter=en_max_iter, tol=1e-3
    )

# Add some standard options for comparison
meta_learners['Ridge_meta_standard'] = Ridge(alpha=1.0)
meta_learners['LinearRegression_meta'] = LinearRegression()

best_meta_mae = float('inf')
best_meta_name = None
best_meta_model = None

print("Evaluating Optuna-enhanced meta-learners:")
for name, meta_model in meta_learners.items():
    # Cross-validate the meta-learner on OOF predictions
    meta_cv_scores = cross_val_score(
        meta_model, level1_oof_preds, y_full,
        cv=5, scoring='neg_mean_absolute_error'
    )
    meta_mae = -meta_cv_scores.mean()
    meta_mae_std = meta_cv_scores.std()
    
    optuna_flag = "🚀" if "optuna" in name else "📊"
    print(f"  {optuna_flag} {name}: MAE = {meta_mae:.4f} (±{meta_mae_std:.4f})")
    
    if meta_mae < best_meta_mae:
        best_meta_mae = meta_mae
        best_meta_name = name
        best_meta_model = meta_model

print(f"\n🏆 Best meta-learner: {best_meta_name}")
print(f"Best meta-learner CV MAE: {best_meta_mae:.4f}")
is_optuna_meta = "optuna" in best_meta_name
print(f"Uses Optuna optimization: {'✅ Yes' if is_optuna_meta else '❌ No'}")

# Train the best meta-learner on all OOF predictions
print(f"\n4. TRAINING FINAL OPTUNA-ENHANCED STACKED MODEL")
print("-" * 60)

final_meta_model = clone(best_meta_model)
final_meta_model.fit(level1_oof_preds, y_full)

# Generate final stacked predictions
stacked_test_pred = final_meta_model.predict(level1_test_preds)

print(f"Optuna-enhanced stacked ensemble test predictions:")
print(f"  Range: {stacked_test_pred.min():.2f} to {stacked_test_pred.max():.2f}")
print(f"  Mean: {stacked_test_pred.mean():.2f}")
print(f"  Std: {stacked_test_pred.std():.2f}")

# Compare with Phase 1 Optuna ensemble
print(f"\n5. COMPARISON WITH PHASE 1 OPTUNA ENSEMBLE")
print("-" * 60)
print(f"Phase 1 Optuna ensemble predictions:")
print(f"  Range: {ensemble_test_pred.min():.2f} to {ensemble_test_pred.max():.2f}")
print(f"  Mean: {ensemble_test_pred.mean():.2f}")
print(f"  Std: {ensemble_test_pred.std():.2f}")

print(f"\nPhase 2 Optuna stacked predictions:")
print(f"  Range: {stacked_test_pred.min():.2f} to {stacked_test_pred.max():.2f}")  
print(f"  Mean: {stacked_test_pred.mean():.2f}")
print(f"  Std: {stacked_test_pred.std():.2f}")

# Calculate correlation between Phase 1 and Phase 2 predictions
correlation = np.corrcoef(ensemble_test_pred, stacked_test_pred)[0, 1]
print(f"\nCorrelation between Phase 1 and Phase 2 Optuna ensembles: {correlation:.4f}")

print(f"\nPhase 2 (Optuna-enhanced stacked ensemble):")
print(f"  CV MAE: {best_meta_mae:.4f}")
improvement_vs_phase1 = optimal_mae - best_meta_mae
print(f"  Expected improvement vs Phase 1: {improvement_vs_phase1:.4f}")

if best_meta_mae < optimal_mae:
    print(f"  ✅ Phase 2 shows improvement over Phase 1!")
    print(f"  🚀 Optuna optimization helped both phases!")
else:
    print(f"  ⚠️ Phase 2 CV did not improve Phase 1")
    print(f"  📊 Both benefit from Optuna optimization")
    
print(f"\n🎯 Both ensembles now use Optuna-optimized models!")


3. TRAINING LEVEL 2 META-LEARNER (OPTUNA-ENHANCED)
------------------------------------------------------------
Evaluating Optuna-enhanced meta-learners:
  🚀 Ridge_meta_optuna: MAE = 2.7240 (±0.0500)
  📊 Ridge_meta_conservative: MAE = 2.7240 (±0.0500)
  📊 Ridge_meta_aggressive: MAE = 2.7240 (±0.0500)
  🚀 Lasso_meta_optuna: MAE = 2.7240 (±0.0500)
  🚀 ElasticNet_meta_optuna: MAE = 2.7240 (±0.0500)
  📊 Ridge_meta_standard: MAE = 2.7240 (±0.0500)
  📊 LinearRegression_meta: MAE = 2.7240 (±0.0500)

🏆 Best meta-learner: Ridge_meta_conservative
Best meta-learner CV MAE: 2.7240
Uses Optuna optimization: ❌ No

4. TRAINING FINAL OPTUNA-ENHANCED STACKED MODEL
------------------------------------------------------------
Optuna-enhanced stacked ensemble test predictions:
  Range: 44.84 to 109.48
  Mean: 79.06
  Std: 12.04

5. COMPARISON WITH PHASE 1 OPTUNA ENSEMBLE
------------------------------------------------------------
Phase 1 Optuna ensemble predictions:
  Range: 44.81 to 109.51
  Mean: 79.0

In [20]:
# Create Optuna-enhanced stacked ensemble submission
print(f"\n6. CREATING OPTUNA-ENHANCED STACKED ENSEMBLE SUBMISSION")
print("-" * 70)

# Create submission DataFrame for Optuna-enhanced stacked ensemble
stacked_submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'W': stacked_test_pred
})

# Generate timestamp
timestamp_stacked = datetime.now().strftime("%Y%m%d_%H%M%S")
stacked_submission_filename = f"submission_optuna_stacked_ensemble_{timestamp_stacked}.csv"
stacked_submission_path = SUB_DIR / stacked_submission_filename

# Save submission
stacked_submission_df.to_csv(stacked_submission_path, index=False)

print(f"✅ Optuna-enhanced stacked ensemble submission saved: {stacked_submission_filename}")
print(f"📁 Path: {stacked_submission_path}")
print(f"📊 Predictions shape: {stacked_submission_df.shape}")

# Display first few predictions
print(f"\nFirst 10 predictions:")
print(stacked_submission_df.head(10))

# Final comprehensive summary
print(f"\n7. OPTUNA-ENHANCED STACKED ENSEMBLE SUMMARY")
print("-" * 70)
print(f"Base Models (all Optuna-optimized): {len(stacking_models)}")
for name in model_names_stack:
    optuna_flag = "🚀" if any(x in name.lower() for x in ['optuna', 'best']) else "📊"
    print(f"  {optuna_flag} {name}")

print(f"\nMeta-learner: {best_meta_name}")
meta_optuna_status = "🚀 Uses Optuna optimization" if "optuna" in best_meta_name else "📊 Standard parameters"
print(f"Meta-learner status: {meta_optuna_status}")

print(f"\nExpected Performance:")
print(f"  CV MAE: {best_meta_mae:.4f}")
print(f"  Expected Kaggle improvement vs Phase 1: {improvement_vs_phase1:.4f}")

print(f"\n🎯 PHASE COMPARISON SUMMARY")
print("-" * 70)
print(f"Phase 1 (Weighted): MAE = {optimal_mae:.4f} | Correlation: {correlation:.3f}")
print(f"Phase 2 (Stacked):  MAE = {best_meta_mae:.4f} | Improvement: {improvement_vs_phase1:.4f}")

winner = "Phase 2 (Stacked)" if best_meta_mae < optimal_mae else "Phase 1 (Weighted)"
print(f"🏆 Current leader: {winner}")

print(f"\n✨ Both phases now leverage full Optuna optimization!")
print(f"🚀 Ready to submit the best performing ensemble to Kaggle!")

# Show the optimization journey
print(f"\n📈 OPTIMIZATION JOURNEY")
print("-" * 70)
print(f"1. ✅ Boosting models optimized with Optuna")
print(f"2. ✅ Linear models optimized with Optuna")  
print(f"3. ✅ Phase 1 ensemble uses Optuna models")
print(f"4. ✅ Phase 2 ensemble uses Optuna models + meta-learner")
print(f"5. 🎯 Ready for Kaggle submission with optimized ensembles!")


6. CREATING OPTUNA-ENHANCED STACKED ENSEMBLE SUBMISSION
----------------------------------------------------------------------
✅ Optuna-enhanced stacked ensemble submission saved: submission_optuna_stacked_ensemble_20251005_111103.csv
📁 Path: /home/chrisfkh/sctp-ds-ai/mod3/kaggle_moneyball/submissions/submission_optuna_stacked_ensemble_20251005_111103.csv
📊 Predictions shape: (453, 2)

First 10 predictions:
     ID          W
0  1756  69.289646
1  1282  74.423262
2   351  84.325458
3   421  87.096260
4    57  93.275050
5  1557  97.585979
6   846  79.219717
7  1658  84.070777
8   112  72.882226
9  2075  83.601103

7. OPTUNA-ENHANCED STACKED ENSEMBLE SUMMARY
----------------------------------------------------------------------
Base Models (all Optuna-optimized): 1
  🚀 Best_Linear

Meta-learner: Ridge_meta_conservative
Meta-learner status: 📊 Standard parameters

Expected Performance:
  CV MAE: 2.7240
  Expected Kaggle improvement vs Phase 1: -0.0006

🎯 PHASE COMPARISON SUMMARY
---------